# Public Health of England Nutrient analysis of fruit and vegetables Sampling Report - 2015

Import Libraries

In [1]:
import re
import PyPDF2
import tabula
import pandas as pd
from fuzzywuzzy import process, fuzz

                                                    Convert PDF to text 

Lots of these didn't have weights needed. Section at the end had tables with raw weights which are more useful.

In [2]:
%%time

# The path to the PDF file
pdf_path = r"C:\Users\medekar\Desktop\Product_Weight_Project\Data\fruit_and_vegetable_survey_2015_sampling_report.pdf"

# Dictionary to store extracted information from pages 11 to 1291
extracted_data_dict = {}

# Open the PDF file in binary read mode
with open(pdf_path, "rb") as file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfFileReader(file)

    # Extract data from all pages (pages 11 to 1291)
    start_page = 11
    end_page = min(1291, pdf_reader.numPages)
    for page_num in range(start_page - 1, end_page):
        # Get the current page object
        page = pdf_reader.getPage(page_num)

        # Extract text from the page
        page_text = page.extractText()

        # Store the extracted text in the dictionary
        extracted_data_dict[page_num + 1] = page_text


Xref table not zero-indexed. ID numbers for objects will be corrected.


CPU times: total: 16.6 s
Wall time: 16.7 s


                                                 Look at output page by page

In [3]:
%%time
# Function to display the text on a specific page
def display_text_on_page(extracted_data_dict, page_number):
    if page_number in extracted_data_dict:
        print("Text on Page", page_number, ":\n", extracted_data_dict[page_number])
    else:
        num_pages = len(extracted_data_dict)
        if num_pages > 0:
            print("Page number", page_number, "not found in the extracted data. Valid page numbers are 1 to", num_pages)
        else:
            print("No pages found in the extracted data. Make sure you have executed the code to extract data.")

# Test:
page_number_to_display = 120
display_text_on_page(extracted_data_dict, page_number_to_display)

Text on Page 120 :
 Nutrient analysis survey of fresh and processed fruit and vegetables with respect to fibre  
 
120  
Composite Sample Number:  
 5 
Composite Sample Name:  
 Mangoes, ripe, raw, flesh only  
Sub-sample Number:  
 5.11 
Sub-sample Name:  
 Farm Fresh mango  
Brand:  
 Farm Fresh  
Pack size:  
 2 fruit  
Purchase Date:  
 28.09.15  
Place of Purchase:  
 Booker, Norwich  
Display Until Date:  
  
Best before Date:  
  
Batch Number:  
 #4051 3604 TEX 262  
Packaging:  
 Plastic  
Sample Condition As Purchased:  
 Ambient  
Post Purchase Storage:  
 21ºC controlled temperature room  
 
 
 
 
 
  
CPU times: total: 0 ns
Wall time: 0 ns


                                                Extract relevant Information

In [4]:
%%time
# Function to extract the specified fields from the text

def extract_texts(text):
    # Dictionary to store the extracted fields
    texts_dict = {}

    # Regular expression patterns for each field
    patterns = {
        "Composite Sample Number": r"Composite Sample Number:\s*(.*)",
        "Composite Sample Name": r"Composite Sample Name:\s*(.*)",
        "Sub-sample Number": r"Sub-sample Number:\s*(.*)",
        "Sub-sample Name": r"Sub-sample Name:\s*(.*)",
        "Brand": r"Brand:\s*(.*)",
        "Pack size": r"Pack size:\s*(.*)",
        "Purchase Date": r"Purchase Date:\s*(.*)",
        "Place of Purchase": r"Place of Purchase:\s*(.*)",
        "Display Until Date": r"Display Until Date:\s*(.*)",
        "Best Before Date": r"Best Before Date:\s*(.*)",
        "Batch Number": r"Batch Number:\s*(.*)",
        "Packaging": r"Packaging:\s*(.*)",
        "Sample Condition As Purchased": r"Sample Condition As Purchased:\s*(.*)",
        "Post Purchase Storage": r"Post Purchase Storage:\s*(.*)",
        "Ingredients": r"Ingredients:\s*(.*)",
        "Label claim(s)": r"Label claim\(s\):\s*(.*)",
    }

    for texts, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            texts_dict[texts] = match.group(1)
        else:
            texts_dict[texts] = None

    return texts_dict

# List to store extracted data from all pages
all_extracted_data = []

# Loop through all pages (pages 13 to 2147)
for page_num in range(13, 2148):
    if page_num in extracted_data_dict:
        page_text_to_extract = extracted_data_dict[page_num]
        extracted_texts = extract_texts(page_text_to_extract)
        all_extracted_data.append(extracted_texts)

# Printing the extracted data for all pages
print("Extracted data from all pages:")
#for idx, data in enumerate(all_extracted_data, start=1):
#    print("Page", idx, ":", data)


Extracted data from all pages:
CPU times: total: 15.6 ms
Wall time: 23.9 ms


                                                  Put Into Pandas Dataframe

In [5]:
df = pd.DataFrame(all_extracted_data)
products = df[~df['Composite Sample Name'].isna()]
# Filtering rows where 'Composite Sample Number' contains 'Composite'
filtered_products = products.loc[~products['Composite Sample Number'].str.contains('Composite', case=False, na=False)]

In [6]:
pd.set_option('display.max_rows', None)
filtered_products.drop(['Ingredients','Label claim(s)'], axis=1, inplace=True)

Export Data

In [7]:
# %%time
# filtered_products.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Data\Processed\WeightData_sourced\PHealth_FV2015.csv")

Read In Data

In [8]:
%%time
filtered_products = pd.read_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Data\Processed\WeightData_sourced\PHealth_FV2015.csv", index_col=0)
filtered_products['Sub-sample Number'] = filtered_products['Sub-sample Number'].astype(str)#.head()

CPU times: total: 0 ns
Wall time: 16.9 ms


In [9]:
filtered_products

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
1,1.0,"Kiwi fruit, raw, flesh only",1.2,Tesco kiwi,Tesco,Purchase Date:,19.10.15,"Tesco Extra, Blue Boar Road, Norwich",Use by Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room
3,1.0,"Kiwi fruit, raw, flesh only",1.3,Asda Grower’s Selection ripen at home kiwi fru...,Asda Grower’s Selection,6 fruits,28.09.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site: 903126 776590 NM 113 07:14,Plastic,Ambient,21°C controlled temperature room
5,1.0,"Kiwi fruit, raw, flesh only",1.4,Sainsbury’s New Zealand kiwi fruit,Sainsbury’s,6 fruit,28.09.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,2 10:37 CO153A,Plastic netting,Ambient,21°C controlled temperature room
7,1.0,"Kiwi fruit, raw, flesh only",1.5,Morrisons kiwi fruit,Morrisons,Purchase Date:,28.09.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,LA53700400,Plastic,Ambient,21°C controlled temperature room
9,1.0,"Kiwi fruit, raw, flesh only",1.6,M&S perfectly ripe kiwi fruit,Marks & Spencer,Purchase Date:,28.09.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,Packaging:,Plastic,Ambient,Chilled
11,1.0,"Kiwi fruit, raw, flesh only",1.7,Lidl kiwi fruit,Lidl,Purchase Date:,28.09.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room
13,1.0,"Kiwi fruit, raw, flesh only",1.8,Tesco Everyday Value kiwi,Tesco Everyday Value,6 fruits,28.09.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC: DPT 7551,Mixed material,Ambient,21°C controlled temperature room
15,1.0,"Kiwi fruit, raw, flesh only",1.9,The Co -operative Loved by Us kiwi fruit,The Co -operative Loved by Us,6 fruit,26.10.15,"The Co -operative, Long Stratton",Best before Date:,NaN,Packaging:,Plastic,Chilled,21°C controlled temperature room
17,1.0,"Kiwi fruit, raw, flesh only",1.1,Greengrocer kiwi,Pack size:,Purchase Date:,30.09.15,"Independent greengrocer, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room
19,1.0,"Kiwi fruit, raw, flesh only",1.11,Bidvest Foodservice kiwi,Bidvest Foodservice,x6,13.10.15,Bidvest Foodservice delivery,Best before Date:,NaN,A23966,Plastic,Ambient,21°C controlled temperature room


##### Extract text converted to dataframe does not have accurate figures. Update this table with figures from tables at end of pdf

                                       Sample Preparation Section (Pdf Tables)

In [10]:
%%time
import tabula
import pandas as pd

# Define the path to your PDF file
pdf_path = r"C:\Users\medekar\Desktop\Product_Weight_Project\Data\fruit_and_vegetable_survey_2015_sampling_report.pdf"

# Extract tables with specific structure
tables = tabula.read_pdf(pdf_path, 
                         pages="1292-1331", 
                         guess=False, 
                         lattice=True,
                         pandas_options={"header": 0}
                        )

dataframes = []

# Process each extracted table and save as individual DataFrames
for idx, table in enumerate(tables):
    # Check if the table has the expected structure (number of columns)
    if table.shape[1] == 2:# Adjust based on your actual table structure
        # Append the DataFrame to the list
        dataframes.append(table)
    elif table.shape[1] == 3:
        dataframes.append(table)
    elif table.shape[1] == 4:
        dataframes.append(table)
    elif table.shape[1] == 5:  
        dataframes.append(table) 
    elif table.shape[1] == 6:
        dataframes.append(table)
    elif table.shape[1] == 7:
        dataframes.append(table)
    elif table.shape[1] == 8:
        dataframes.append(table)
        

# Print the contents of each DataFrame and save as individual pandas DataFrames
saved_dataframes = []
for idx, df in enumerate(dataframes):
    print(f"Table {idx + 1}:")
    print(df)
    print("-----------------------------")
    
    # Save the DataFrame as an individual pandas DataFrame
    saved_dataframes.append(df)

Table 1:
   Sub-sample\rnumber  Average\rweight of a\rwhole kiwi\rfruit (g)  \
0                 1.1                                         77.3   
1                 1.2                                         95.8   
2                 1.3                                         85.1   

   Average\rweight of\rskin (g)  Average\rweight of\rends (g)  Edible\rportion  
0                          13.0                           8.3             0.72  
1                          16.0                           9.1             0.74  
2                          14.7                           9.1             0.72  
-----------------------------
Table 2:
    1.4   75.3  11.7   7.5  0.75
0  1.50   72.6  10.9   4.0  0.79
1  1.60  110.7  16.5   9.0  0.77
2  1.70   97.1  16.9  11.4  0.71
3  1.80   69.3  11.9  12.0  0.65
4  1.90   83.4  14.3   8.5  0.73
5  1.10   80.4  13.1  12.1  0.69
6  1.11   68.5  10.3   7.0  0.75
-----------------------------
Table 3:
    Sub-sample\rnumber  Average\rweight of a

Filtering db

                                                         Kiwi

Dep of Health Text pdf

In [11]:
rawKiw = filtered_products[filtered_products['Composite Sample Name'].str.contains('Kiwi')]
rawKiw

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
1,1.0,"Kiwi fruit, raw, flesh only",1.2,Tesco kiwi,Tesco,Purchase Date:,19.10.15,"Tesco Extra, Blue Boar Road, Norwich",Use by Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room
3,1.0,"Kiwi fruit, raw, flesh only",1.3,Asda Grower’s Selection ripen at home kiwi fru...,Asda Grower’s Selection,6 fruits,28.09.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site: 903126 776590 NM 113 07:14,Plastic,Ambient,21°C controlled temperature room
5,1.0,"Kiwi fruit, raw, flesh only",1.4,Sainsbury’s New Zealand kiwi fruit,Sainsbury’s,6 fruit,28.09.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,2 10:37 CO153A,Plastic netting,Ambient,21°C controlled temperature room
7,1.0,"Kiwi fruit, raw, flesh only",1.5,Morrisons kiwi fruit,Morrisons,Purchase Date:,28.09.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,LA53700400,Plastic,Ambient,21°C controlled temperature room
9,1.0,"Kiwi fruit, raw, flesh only",1.6,M&S perfectly ripe kiwi fruit,Marks & Spencer,Purchase Date:,28.09.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,Packaging:,Plastic,Ambient,Chilled
11,1.0,"Kiwi fruit, raw, flesh only",1.7,Lidl kiwi fruit,Lidl,Purchase Date:,28.09.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room
13,1.0,"Kiwi fruit, raw, flesh only",1.8,Tesco Everyday Value kiwi,Tesco Everyday Value,6 fruits,28.09.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC: DPT 7551,Mixed material,Ambient,21°C controlled temperature room
15,1.0,"Kiwi fruit, raw, flesh only",1.9,The Co -operative Loved by Us kiwi fruit,The Co -operative Loved by Us,6 fruit,26.10.15,"The Co -operative, Long Stratton",Best before Date:,NaN,Packaging:,Plastic,Chilled,21°C controlled temperature room
17,1.0,"Kiwi fruit, raw, flesh only",1.1,Greengrocer kiwi,Pack size:,Purchase Date:,30.09.15,"Independent greengrocer, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room
19,1.0,"Kiwi fruit, raw, flesh only",1.11,Bidvest Foodservice kiwi,Bidvest Foodservice,x6,13.10.15,Bidvest Foodservice delivery,Best before Date:,NaN,A23966,Plastic,Ambient,21°C controlled temperature room


In [12]:
rawKiw['Sub-sample Number'] = rawKiw['Sub-sample Number'].astype(str)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\1273488456.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawKiw['Sub-sample Number'] = rawKiw['Sub-sample Number'].astype(str)


Dep of Health Tables pdf

##### Sample 1: Kiwi fruit, raw, flesh only

Kiwis were peeled and the ends removed before sampling. Each sub-sample was taken from
eight kiwis.


#### Average weight of kiwi fruit = 83.8g

In [13]:
Ki1 = saved_dataframes[0]#.drop(['Average weight\rof other inedible\rparts (g)','Average edible\rportion'],axis=1)
Ki1

,Sub-sample\rnumber,Average\rweight of a\rwhole kiwi\rfruit (g),Average\rweight of\rskin (g),Average\rweight of\rends (g),Edible\rportion
0,1.1,77.3,13.0,8.3,0.72
1,1.2,95.8,16.0,9.1,0.74
2,1.3,85.1,14.7,9.1,0.72


In [14]:
Ki2 = saved_dataframes[1]
Ki2

,1.4,75.3,11.7,7.5,0.75
0,1.50,72.6,10.9,4.0,0.79
1,1.60,110.7,16.5,9.0,0.77
2,1.70,97.1,16.9,11.4,0.71
3,1.80,69.3,11.9,12.0,0.65
4,1.90,83.4,14.3,8.5,0.73
5,1.10,80.4,13.1,12.1,0.69
6,1.11,68.5,10.3,7.0,0.75


In [15]:
Ki1['Sub-sample\rnumber'] = Ki1['Sub-sample\rnumber'].astype(str)

In [16]:
Ki2.columns = Ki1.columns

In [17]:
Ki2['Sub-sample\rnumber'] = Ki2['Sub-sample\rnumber'].astype(str)

Join both tables to crate one talbe for potatoes

In [18]:
Kiwi = pd.concat([Ki1,Ki2],ignore_index=True)
Kiwi.drop(8, inplace=True)

In [19]:
Kiwi

,Sub-sample\rnumber,Average\rweight of a\rwhole kiwi\rfruit (g),Average\rweight of\rskin (g),Average\rweight of\rends (g),Edible\rportion
0,1.1,77.3,13.0,8.3,0.72
1,1.2,95.8,16.0,9.1,0.74
2,1.3,85.1,14.7,9.1,0.72
3,1.5,72.6,10.9,4.0,0.79
4,1.6,110.7,16.5,9.0,0.77
5,1.7,97.1,16.9,11.4,0.71
6,1.8,69.3,11.9,12.0,0.65
7,1.9,83.4,14.3,8.5,0.73
9,1.11,68.5,10.3,7.0,0.75


Match sub sample to filtered product database and update filtered product database to with weight in pot 1

In [20]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawkidf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = Kiwi[Kiwi['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawkidf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole kiwi\rfruit (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
rawKiw['Pack size'] = rawKiw.apply(update_pack_size, axis=1)


C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\2577845584.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawKiw['Pack size'] = rawKiw.apply(update_pack_size, axis=1)


Checking if this worked well

In [21]:
rawKiw.tail()

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
11,1.0,"Kiwi fruit, raw, flesh only",1.7,Lidl kiwi fruit,Lidl,97.1,28.09.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room
13,1.0,"Kiwi fruit, raw, flesh only",1.8,Tesco Everyday Value kiwi,Tesco Everyday Value,69.3,28.09.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC: DPT 7551,Mixed material,Ambient,21°C controlled temperature room
15,1.0,"Kiwi fruit, raw, flesh only",1.9,The Co -operative Loved by Us kiwi fruit,The Co -operative Loved by Us,83.4,26.10.15,"The Co -operative, Long Stratton",Best before Date:,NaN,Packaging:,Plastic,Chilled,21°C controlled temperature room
17,1.0,"Kiwi fruit, raw, flesh only",1.1,Greengrocer kiwi,Pack size:,77.3,30.09.15,"Independent greengrocer, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room
19,1.0,"Kiwi fruit, raw, flesh only",1.11,Bidvest Foodservice kiwi,Bidvest Foodservice,68.5,13.10.15,Bidvest Foodservice delivery,Best before Date:,NaN,A23966,Plastic,Ambient,21°C controlled temperature room


Rename column header and export file

In [22]:
rawKiw.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
rawKiw.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Kiwi1.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\2799310280.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rawKiw.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


##### Sample 2: Plums, dessert, raw, flesh and skin

Plums were rinsed in Analar water and dried. Stones were removed before sampling. Each
sub-sample was randomly taken from 4-8 plums.


##### Average weight of a plum and stone: 76.8g

In [23]:
plum = saved_dataframes[2]
#plum

In [24]:
plum['Sub-sample\rnumber'] = plum['Sub-sample\rnumber'].astype(str)

In [25]:
plum

,Sub-sample\rnumber,Average\rweight of a\rwhole plum\r(g),Average\rweight of\rstone (g),Edible\rportion
0,2.1,129.0,1.8,0.99
1,2.2,76.7,1.1,0.99
2,2.3,70.7,1.0,0.99
3,2.4,77.5,3.7,0.95
4,2.5,83.0,2.8,0.97
5,2.6,77.7,2.4,0.97
6,2.7,77.0,1.4,0.98
7,2.8,54.6,2.6,0.95
8,2.9,69.1,1.6,0.98
9,2.1,39.3,1.7,0.96


In [26]:
plum.rename(columns={'Average\rweight of a\rwhole plum\r(g)': 'Sample Weight'}, inplace = True)
plum.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\plum.csv")

##### Sample 3: Raspberries, raw

Raspberries were sampled as purchased and not rinsed. Experimentation showed that, when
rinsed, they were difficult to dry and had a tendency to disintegrate. Each sub-sample consisted
of 20-30 randomly selected raspberries.


#### Average weight of a raspberry = 4.9g

Use pack size for such fruits


Dep of Health text pdf

In [27]:
rberry = filtered_products[filtered_products['Composite Sample Name'].str.contains('Raspberries')]
rberry

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
43,3.0,"Raspberries, raw",3.1,Tesco British raspberries,Tesco,150g,11.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC: KG45,Plastic,Chilled,Chilled
45,3.0,"Raspberries, raw",3.2,Tesco raspberries,Tesco,150g,02.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC: BY05 L20327,Plastic,Chilled,Chilled
47,3.0,"Raspberries, raw",3.3,Asda Grower’s Selection raspberries,Asda Grower’s Selection,150g,02.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: 785263 Site: 900087 10:04,Plastic,Chilled,Chilled
49,3.0,"Raspberries, raw",3.4,Sainsbury’s Taste the Difference British raspb...,Sainsbury’s Taste the Difference,150g,11.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,A0800 DD,Plastic,Chilled,Chilled
51,3.0,"Raspberries, raw",3.5,Morrisons raspberries,Morrisons,150g,11.08.15,"Morrisons, Norwich",Best before Date:,NaN,1126HHD319 T49007/0714,Plastic,Chilled,Chilled
53,3.0,"Raspberries, raw",3.6,Essential Waitrose raspberries,Essential Waitrose,160g,05.11.15,"Waitrose, Wymondham",Best before Date:,NaN,Supplier: 102WSP 09.14,Plastic,Chilled,Chilled
55,3.0,"Raspberries, raw",3.7,Nature’s Fruit raspberries,Aldi Nature’s Fruit,150g,11.08.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,Supplier code: 02944/NHR 3303 (3305C),Plastic,Chilled,Chilled
57,3.0,"Raspberries, raw",3.8,Asda Grower’s Selection raspberries,Asda Grower’s Selection,150g,02.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: 785654 Site:900709 19:06,Plastic,Chilled,Chilled
59,3.0,"Raspberries, raw",3.9,The Co -operative Loved by Us Scottish raspber...,The Co -operative Loved by Us,150g,11.08.15,"The Co -operative, Unthank Road, Norwich",Best before Date:,NaN,M150/TO,Plastic,Chilled,Chilled
61,3.0,"Raspberries, raw",nan,Greengrocer raspberries,Pack size:,Purchase Date:,11.08.15,"Independent greengrocer, Norwich",Best before Date:,NaN,Packaging:,Plastic,Ambient,Chilled


Deo of Health tables pdf

##### Raspberry

In [28]:
berry1 = saved_dataframes[3]
berry1

,Sub-sample\rnumber,Average weight of a\rraspberry (g)
0,3.1,3.9
1,3.2,4.9


In [29]:
berry2 = saved_dataframes[4]#[10:19].drop(['Average weight\rof other inedible\rparts (g)','Average edible\rportion'],axis=1)
berry2

,3.3,3.1
0,3.40,7.3
1,3.50,3.7
2,3.60,3.8
3,3.70,5.7
4,3.80,5.7
5,3.90,5.4
6,3.10,6.5
7,3.11,3.8


In [30]:
berry1['Sub-sample\rnumber'] = berry1['Sub-sample\rnumber'].astype(str)

In [31]:
berry2.columns = berry1.columns

In [32]:
berry2['Sub-sample\rnumber'] = berry2['Sub-sample\rnumber'].astype(str)

Join both tables to crate one talbe for potatoes

In [33]:
raspberry = pd.concat([berry1,berry2],ignore_index=True)
raspberry.drop(8, inplace=True)

In [34]:
raspberry

,Sub-sample\rnumber,Average weight of a\rraspberry (g)
0,3.1,3.9
1,3.2,4.9
2,3.4,7.3
3,3.5,3.7
4,3.6,3.8
5,3.7,5.7
6,3.8,5.7
7,3.9,5.4
9,3.11,3.8


Matching Pot 2 

In [35]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawrasdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = raspberry[raspberry['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawrasdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average weight of a\rraspberry (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
rberry['Pack size'] = rberry.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\280650170.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rberry['Pack size'] = rberry.apply(update_pack_size, axis=1)


In [36]:
rberry.tail()

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
55,3.0,"Raspberries, raw",3.7,Nature’s Fruit raspberries,Aldi Nature’s Fruit,5.7,11.08.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,Supplier code: 02944/NHR 3303 (3305C),Plastic,Chilled,Chilled
57,3.0,"Raspberries, raw",3.8,Asda Grower’s Selection raspberries,Asda Grower’s Selection,5.7,02.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: 785654 Site:900709 19:06,Plastic,Chilled,Chilled
59,3.0,"Raspberries, raw",3.9,The Co -operative Loved by Us Scottish raspber...,The Co -operative Loved by Us,5.4,11.08.15,"The Co -operative, Unthank Road, Norwich",Best before Date:,NaN,M150/TO,Plastic,Chilled,Chilled
61,3.0,"Raspberries, raw",nan,Greengrocer raspberries,Pack size:,Purchase Date:,11.08.15,"Independent greengrocer, Norwich",Best before Date:,NaN,Packaging:,Plastic,Ambient,Chilled
63,3.0,"Raspberries, raw",3.11,Berries R Us Portuguese raspberries,Berries R Us,3.8,24.08.15,Bidvest Foodservice delivery,Best before Date:,NaN,Packaging:,Plastic,Chilled,Chilled


Export

In [37]:
rberry.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
rberry.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\raspberry.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\1494650560.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rberry.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


                                                      Pineapple

Doh Text pdf

##### Sample 4: Pineapple, fresh, raw, flesh only

The top, bottom, core and skin were removed from whole pineapples. Each sub-sample was
taken from quarters of two pineapples.


##### Average weight of a pineapple = 1198.4g

In [38]:
pine = filtered_products[filtered_products['Composite Sample Name'].str.contains('Pineapple')]
pine

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
65,4.0,"Pineapple, fresh, raw, flesh only",4.1,Tesco pineapple chunks,Tesco,400g,28.09.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,L5 17:33 SC3660,Plastic,Chilled,Chilled
67,4.0,"Pineapple, fresh, raw, flesh only",4.2,Tesco pineapple,Tesco,Purchase Date:,28.09.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC: PGP E,Loose,Ambient,21ºC c ontrolled temperature room
69,4.0,"Pineapple, fresh, raw, flesh only",4.3,Asda Grower’s Selection pineapple chunks,Asda Grower’s Selection,400g,28.09.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,175 P11,Plastic,Chilled,Chilled
71,4.0,"Pineapple, fresh, raw, flesh only",4.4,Sainsbury’s pineapple slices,Sainsbury’s,350g,28.09.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,B1337A 7282199 09.14,Plastic,Chilled,Chilled
73,4.0,"Pineapple, fresh, raw, flesh only",4.5,Morrisons pineapple,Morrisons,180g,28.09.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1424/3 1 16:17 1424 T44682/0114,Plastic,Chilled,Chilled
75,4.0,"Pineapple, fresh, raw, flesh only",4.6,M&S perfectly ripe large golden pineapple,Marks & Spencer,Purchase Date:,28.09.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,4929,Loose,Ambient,21ºC c ontrolled temperature room
77,4.0,"Pineapple, fresh, raw, flesh only",4.7,Natura Farms gold pineapple,Lidl Natura Farms,Purchase Date:,28.09.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC c ontrolled temperature room
79,4.0,"Pineapple, fresh, raw, flesh only",4.8,Sainsbury’s basics pineapple,Sainsbury’s basics,Purchase Date:,05.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,F0237C 1092585 09.14,Loose; tag - mixed material,Ambient,21ºC c ontrolled temperature room
81,4.0,"Pineapple, fresh, raw, flesh only",4.9,The Co -operative Loved by Us pineapple,The Co -operative Loved by Us,Purchase Date:,06.10.15,"Co-op, Wymondham",Best before Date:,NaN,Supplied by: M206C KEGE12,Loose; tag - cardboard,Ambient,21°C controlled temperature room
83,4.0,"Pineapple, fresh, raw, flesh only",4.1,Del Monte Gold extra sweet pineapple,Del Monte,Purchase Date:,28.09.15,"Independent greengrocer, Norwich",Best before Date:,NaN,10301465,Loose; tag - cardboard,Ambient,21ºC controlled temperature room


In [39]:
pine['Sub-sample Number'] = pine['Sub-sample Number'].astype(str)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\2937672426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pine['Sub-sample Number'] = pine['Sub-sample Number'].astype(str)


Doh Text pdf

In [40]:
pineapp = saved_dataframes[5]
pineapp

,Sub-\rsample\rnumber,Average\rweight of a\rwhole\rpineapple(g),Average\rweight\rof top\r(g),Average\rweight\rof\rbottom\r(g),Average\rweight\rof skin\r(g),Average\rweight\rof core\r(g),Average\rweight\rof slice/\rchunk\r(g),Edible\rportion
0,4.10,N/A,N/A,N/A,N/A,N/A,11.3,N/A
1,4.20,1265.6,287.7,49.5,265.4,68.7,N/A,0.47
2,4.30,N/A,N/A,N/A,N/A,N/A,10.0,N/A
3,4.40,N/A,N/A,N/A,N/A,N/A,34.2,N/A
4,4.50,N/A,N/A,N/A,N/A,N/A,13.5,N/A
5,4.60,1537.0,228.2,55.3,355.7,90.6,N/A,0.53
6,4.70,1182.6,344.8,52.2,255.1,120.5,N/A,0.35
7,4.80,885.1,51.5,50.8,285.9,102.3,N/A,0.45
8,4.90,1130.8,232.6,57.2,309.7,98.2,N/A,0.38
9,4.10,1157.8,224.5,58.3,298.1,137.8,N/A,0.38


In [41]:
pineapp['Sub-\rsample\rnumber'] = pineapp['Sub-\rsample\rnumber'].astype(str)

Matching 

In [42]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawpindf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = pineapp[pineapp['Sub-\rsample\rnumber'].apply(lambda x: regex_matching(rawpindf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole\rpineapple(g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
pine['Pack size'] = pine.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\3980124064.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pine['Pack size'] = pine.apply(update_pack_size, axis=1)


In [43]:
pine.tail()

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
77,4.0,"Pineapple, fresh, raw, flesh only",4.7,Natura Farms gold pineapple,Lidl Natura Farms,1182.6,28.09.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC c ontrolled temperature room
79,4.0,"Pineapple, fresh, raw, flesh only",4.8,Sainsbury’s basics pineapple,Sainsbury’s basics,885.1,05.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,F0237C 1092585 09.14,Loose; tag - mixed material,Ambient,21ºC c ontrolled temperature room
81,4.0,"Pineapple, fresh, raw, flesh only",4.9,The Co -operative Loved by Us pineapple,The Co -operative Loved by Us,1130.8,06.10.15,"Co-op, Wymondham",Best before Date:,NaN,Supplied by: M206C KEGE12,Loose; tag - cardboard,Ambient,21°C controlled temperature room
83,4.0,"Pineapple, fresh, raw, flesh only",4.1,Del Monte Gold extra sweet pineapple,Del Monte,N/A,28.09.15,"Independent greengrocer, Norwich",Best before Date:,NaN,10301465,Loose; tag - cardboard,Ambient,21ºC controlled temperature room
85,4.0,"Pineapple, fresh, raw, flesh only",4.11,Del Monte Gold extra sweet pineapple,Del Monte,1229.9,24.08.15,Bidvest Foodservice delivery,Best before Date:,NaN,10301465,Loose; tag - cardboard,Ambient,21ºC controlled temperature room


Rename and export

In [44]:
pine.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
pine.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\pineapple.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\3296130620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pine.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


Doh Text pdf

##### Sample 5: Mangoes, ripe, raw, flesh only

Whole mangoes were peeled and the flesh cut off until the stone was reached. Each subsample
was taken from 2-3 mangoes.


##### Average weight of a mango = 375.2g

In [45]:
mang = filtered_products[filtered_products['Composite Sample Name'].str.contains('Mango')]
mang

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
87,5.0,"Mangoes , ripe, raw, flesh only",5.1,Tesco mango,Tesco,3 fruits,07.12.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Supplier code: SC TR001,Plastic,Ambient,Chilled
89,5.0,"Mangoes, ripe, raw, flesh only",5.2,Tesco mango chunks,Tesco,275g,07.12.15,"Tesco, Harford Bridges, Norwich",Use By Date:,NaN,03:42 KD01,Plastic,Chilled,Chilled
91,5.0,"Mangoes, ripe, raw, flesh only",5.3,Asda Grower’s Selection giant mango,Asda Grower’s Selection,Purchase Date:,28.09.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,909998 ASP1419466,Loose,Ambient,21ºC controlled temperature room
93,5.0,"Mangoes, ripe, raw, flesh only",5.4,Sainsbury’s ripe & ready mangoes,Sainsbury’s,2 fruit,28.09.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,F0224M 1291681 11.14,Plastic,Ambient,21ºC controlled temperature room
95,5.0,"Mangoes, ripe, raw, flesh only",5.5,Morrisons mango,Morrisons,Purchase Date:,07.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,275964 TH1CON1339 05:56 ED LA53702020,Loose,Ambient,21ºC controlled temperature room
97,5.0,"Mangoes, ripe, raw, flesh only",5.6,M&S large mango,Marks & Spencer,Purchase Date:,28.09.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC4808,Loose,Ambient,21ºC controlled temperature room
101,5.0,"Mangoes, ripe, raw, flesh only",5.8,Asda Smart Price mangos,Asda Smart Price,4 fruit,28.09.15,"Asda, Boundary Road, Norwich",03.10.15,NaN,PO.442956,Plastic,Ambient,21ºC controlled temperature room
103,5.0,"Mangoes, ripe, raw, flesh only",5.9,The Co -operative Loved by Us mango chunks,The Co -operative Loved by Us,250g,07.12.15,"The Co -operative, Long Stratton",Best before Date:,NaN,M252 Line F1 00:33 P14455/1/3,Plastic,Chilled,Chilled
105,5.0,"Mangoes, ripe, raw, flesh only",5.1,Andalor Mango,Anadalor,Purchase Date:,13.10.15,"Independent farm shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
107,5.0,"Mangoes, ripe, raw, flesh only",5.11,Farm Fresh mango,Farm Fresh,2 fruit,28.09.15,"Booker, Norwich",Best before Date:,NaN,#4051 3604 TEX 262,Plastic,Ambient,21ºC controlled temperature room


In [46]:
mang['Sub-sample Number'] = mang['Sub-sample Number'].astype(str)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\497862350.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mang['Sub-sample Number'] = mang['Sub-sample Number'].astype(str)


Doh Table pdf 

In [47]:
mango2 = saved_dataframes[7]

In [48]:
mango2.rename(columns={'Unnamed: 0':'Sub-sample\rnumber', 'whole\rmango (g)':'Average\rweight of a whole\rmango (g)',
                       'skin (g)':'Average\rweight of skin (g)', 'stone (g)':'Average\rweight of.1 stone (g)', 
                      'chunk (g)':'Average\rweight of a.1 chunk (g)', 'Unnamed: 1':'Edible\rportion'}, inplace= True)

In [49]:
mango2

,Sub-sample\rnumber,Average\rweight of a whole\rmango (g),Average\rweight of skin (g),Average\rweight of.1 stone (g),Average\rweight of a.1 chunk (g),Edible\rportion
0,5.10,321.7,67.6,49.1,N/A,0.64
1,5.20,N/A,N/A,N/A,20.48,N/A
2,5.30,280.9,56.5,35.4,N/A,0.67
3,5.40,308.2,75.9,42.0,N/A,0.62
4,5.50,285.0,52.6,43.5,N/A,0.66
5,5.60,528.5,110.4,72.8,N/A,0.65
6,5.70,517.9,121.3,67.3,N/A,0.64
7,5.80,385.5,80.5,74.0,N/A,0.60
8,5.90,N/A,N/A,N/A,28.16,N/A
9,5.10,292.5,45.8,76.6,N/A,0.58


In [50]:
mango2['Sub-sample\rnumber'] = mango2['Sub-sample\rnumber'].astype(str)

Matching

In [51]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawmandf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = mango2[mango2['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawmandf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a whole\rmango (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
mang['Pack size'] = mang.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\3763352867.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mang['Pack size'] = mang.apply(update_pack_size, axis=1)


In [52]:
mang.tail()

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
97,5.0,"Mangoes, ripe, raw, flesh only",5.6,M&S large mango,Marks & Spencer,528.5,28.09.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC4808,Loose,Ambient,21ºC controlled temperature room
101,5.0,"Mangoes, ripe, raw, flesh only",5.8,Asda Smart Price mangos,Asda Smart Price,385.5,28.09.15,"Asda, Boundary Road, Norwich",03.10.15,NaN,PO.442956,Plastic,Ambient,21ºC controlled temperature room
103,5.0,"Mangoes, ripe, raw, flesh only",5.9,The Co -operative Loved by Us mango chunks,The Co -operative Loved by Us,N/A,07.12.15,"The Co -operative, Long Stratton",Best before Date:,NaN,M252 Line F1 00:33 P14455/1/3,Plastic,Chilled,Chilled
105,5.0,"Mangoes, ripe, raw, flesh only",5.1,Andalor Mango,Anadalor,321.7,13.10.15,"Independent farm shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
107,5.0,"Mangoes, ripe, raw, flesh only",5.11,Farm Fresh mango,Farm Fresh,456.9,28.09.15,"Booker, Norwich",Best before Date:,NaN,#4051 3604 TEX 262,Plastic,Ambient,21ºC controlled temperature room


rename and export

In [53]:
mang.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
mang.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\mango.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\780150775.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mang.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


Doh tables pdf

##### Sample 6: Nectarines, yellow flesh, raw, flesh only
Nectarines were rinsed in Analar water, dried and the flesh removed from the stone. Each subsample
was taken from quarters of four nectarines.

##### Average weight of a nectarine = 131.6g


In [54]:
nect = saved_dataframes[8]

In [55]:
nect['Sub-sample\rnumber'] = nect['Sub-sample\rnumber'].astype(str)
nect

,Sub-sample\rnumber,Average\rweight of a\rwhole\rnectarine\r(g),Average\rweight of\rstone (g),Edible\rportion
0,6.1,118.4,12.3,0.90
1,6.2,92.0,8.8,0.90
2,6.3,227.1,15.5,0.93
3,6.4,130.8,10.4,0.92
4,6.5,144.0,11.2,0.92
5,6.6,118.7,11.2,0.91
6,6.7,153.4,8.4,0.95
7,6.8,119.5,5.1,0.96
8,6.9,156.7,9.9,0.94
9,6.1,96.7,6.4,0.93


Doh text pdf 

In [56]:
necta = filtered_products[filtered_products['Composite Sample Name'].str.contains('Nectarines')]
necta

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
109,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.1,Tesco yellow flesh nectarines,Tesco,Purchase Date:,14.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Supplier code: GZ08,Plastic with netting,Chilled,21ºC controlled temperature room
111,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.2,Tesco yellow flesh nectarines,Tesco,5 fruit s,19.10.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC DPT 8256,Plastic with netting,Ambient,21ºC controlled temperature room
113,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.3,Asda Grower’s Selection nectarines,Asda Grower’s Selection,1kg,17.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,T219/20915081013 Site: 907790 PO: 420415,Plastic with netting,Chilled,21ºC controlled temperature room
115,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.4,Sainsbury’s nectarines,Sainsbury’s,4 fruit,14.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,M0249GU 517751 10.13,Plastic with netting,Chilled,21ºC controlled temperature room
117,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.5,Morrisons nectarines,Morrisons,Min 4 fruit,17.08.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,LA53700209 208915 TH1 SOR2 226 12:24 MS,Plastic with netting,Chilled,21ºC controlled temperature room
119,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.6,M&S yellow flesh nectarines,Marks & Spencer,Purchase Date:,26.10.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",NaN,NaN,1826,Plastic with paper tray,Chilled,Chilled
121,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.7,Nature’s Fruit nectarines,Aldi Nature’s Fruit,4 fruit,17.08.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,3403 (3405C) 00200 TZZ ALR1404010 25341300,Plastic with card tray,Chilled,Chilled
123,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.8,The Co -operative Loved by Us nectarines,The Co -operative Loved by Us,4 fruit s,17.08.15,"The Co -operative, Earlham Green Lane, Norwich",Best before Date:,NaN,M060AQ,Plastic with netting,Chilled,21ºC controlled temperature room
125,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.9,Gog de Magog nectarines,Gog de Magog,Purchase Date:,19.08.15,"Independent market stall, Norwich M arket",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
127,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.1,Morrisons nectarines,Morrisons,5 fruit,19.10.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1230/UUK LA53700209,Plastic with netting,Ambient,21ºC controlled temperature room


In [57]:
necta['Sub-sample Number'] = necta['Sub-sample Number'].astype(str)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\2405865086.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  necta['Sub-sample Number'] = necta['Sub-sample Number'].astype(str)


Match

In [58]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawnetdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = nect[nect['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawnetdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole\rnectarine\r(g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
necta['Pack size'] = necta.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\693177888.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  necta['Pack size'] = necta.apply(update_pack_size, axis=1)


In [59]:
necta.tail()

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
121,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.7,Nature’s Fruit nectarines,Aldi Nature’s Fruit,153.4,17.08.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,3403 (3405C) 00200 TZZ ALR1404010 25341300,Plastic with card tray,Chilled,Chilled
123,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.8,The Co -operative Loved by Us nectarines,The Co -operative Loved by Us,119.5,17.08.15,"The Co -operative, Earlham Green Lane, Norwich",Best before Date:,NaN,M060AQ,Plastic with netting,Chilled,21ºC controlled temperature room
125,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.9,Gog de Magog nectarines,Gog de Magog,156.7,19.08.15,"Independent market stall, Norwich M arket",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
127,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.1,Morrisons nectarines,Morrisons,118.4,19.10.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1230/UUK LA53700209,Plastic with netting,Ambient,21ºC controlled temperature room
129,6.0,"Nectarines, yellow flesh, raw, flesh and skin",6.11,Tesco yellow flesh nectarines,Tesco,90.0,26.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC: DPT 8388,Plastic with netting,Chilled,21ºC controlled temperature room


Rename and Export

In [60]:
necta.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
necta.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\nectarines.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\243620218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  necta.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


Doh tables pdf

##### Sample 7: Cherries, raw
Cherries were rinsed in Analar water, dried and the stones (and stalks where present)
removed. Each sub-sample consisted of 30-40 halves or whole cherries.

##### Average weight of a cherry = 9.8g

In [61]:
cheer = saved_dataframes[9]
cheer

,Sub-sample\rnumber,Average\rweight of a\rwhole\rcherry (g),Average\rweight of\rstalk (g),Average\rweight of\rstone (g),Edible\rportion
0,7.10,10.7,0.1,0.7,0.92
1,7.20,14.9,0.2,0.8,0.94
2,7.30,8.8,0.1,0.8,0.90
3,7.40,8.7,0.0,0.8,0.90
4,7.50,9.6,0.1,0.8,0.91
5,7.60,7.6,0.0,0.7,0.90
6,7.70,13.0,0.1,0.7,0.94
7,7.80,7.6,0.1,0.6,0.91
8,7.90,9.0,0.0,0.7,0.92
9,7.10,9.8,0.1,0.8,0.91


In [62]:
cheer['Sub-sample\rnumber'] = cheer['Sub-sample\rnumber'].astype(str)

Doh text pdf

In [63]:
cherry = filtered_products[filtered_products['Composite Sample Name'].str.contains('Cherries')]
cherry

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
131,7.0,"Cherries, raw",7.1,Tesco cherries,Tesco,Purchase Date:,11.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Supplier code: KQ33 PLU 4045,Plastic,Chilled,Chilled
133,7.0,"Cherries, raw",7.2,Tesco British cherries,Tesco,250g,11.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC: KG56,Plastic,Chilled,Chilled
135,7.0,"Cherries, raw",7.3,Asda Grower’s Selection sweet & fragrant cherr...,Asda Grower’s Selection,425g,11.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: L72541 Site: 427032 ASP1419288,Plastic,Chilled,Chilled
137,7.0,"Cherries, raw",7.4,Sainsbury’s British cherries,Sainsbury’s,250g,11.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,A0433W 05:26 W2 11.14 7662145,Plastic,Chilled,Chilled
139,7.0,"Cherries, raw",7.5,Morrisons cherries,Morrisons,225g,11.08.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1954/JOS T47513/0514 1954,Plastic,Chilled,Chilled
141,7.0,"Cherries, raw",7.6,Waitrose lusciously sweet & juicy cherries,Waitrose,180g,18.11.15,"Waitrose, Wymondham",20.11.15,NaN,Supplier: 347AAO L99646,Plastic,Chilled,Chilled
143,7.0,"Cherries, raw",7.7,Lidl cherries,Lidl,Purchase Date:,18.11.15,"Lidl, Drayton Road, Norwich",NaN,NaN,Packaging:,Loose,Chilled,Chilled
145,7.0,"Cherries, raw",7.8,The Co -operative Loved by Us cherries,The Co-operative Loved by Us,200g,12.08.15,"The Co -operative, Unthank Road, Norwich",Best before Date:,NaN,Supplied by: M150/TS P6729F/2/2,Plastic,Chilled,Chilled
147,7.0,"Cherries, raw",7.9,Sainsbury’s cherries,Sainsbury’s,200g,16.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,A0800 X2,Plastic,Chilled,Chilled
149,7.0,"Cherries, raw",7.1,Market cherries,Pack size:,Purchase Date:,12.08.15,"Independent market stall, Norwich M arket",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled


In [64]:
cherry['Sub-sample Name'] = cherry['Sub-sample Name'].astype(str)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\183181198.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cherry['Sub-sample Name'] = cherry['Sub-sample Name'].astype(str)


Match

In [65]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawcherdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = cheer[cheer['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawcherdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole\rcherry (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
cherry['Pack size'] = cherry.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\2480522264.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cherry['Pack size'] = cherry.apply(update_pack_size, axis=1)


In [66]:
cherry

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
131,7.0,"Cherries, raw",7.1,Tesco cherries,Tesco,10.7,11.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Supplier code: KQ33 PLU 4045,Plastic,Chilled,Chilled
133,7.0,"Cherries, raw",7.2,Tesco British cherries,Tesco,14.9,11.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC: KG56,Plastic,Chilled,Chilled
135,7.0,"Cherries, raw",7.3,Asda Grower’s Selection sweet & fragrant cherr...,Asda Grower’s Selection,8.8,11.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: L72541 Site: 427032 ASP1419288,Plastic,Chilled,Chilled
137,7.0,"Cherries, raw",7.4,Sainsbury’s British cherries,Sainsbury’s,8.7,11.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,A0433W 05:26 W2 11.14 7662145,Plastic,Chilled,Chilled
139,7.0,"Cherries, raw",7.5,Morrisons cherries,Morrisons,9.6,11.08.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1954/JOS T47513/0514 1954,Plastic,Chilled,Chilled
141,7.0,"Cherries, raw",7.6,Waitrose lusciously sweet & juicy cherries,Waitrose,7.6,18.11.15,"Waitrose, Wymondham",20.11.15,NaN,Supplier: 347AAO L99646,Plastic,Chilled,Chilled
143,7.0,"Cherries, raw",7.7,Lidl cherries,Lidl,13.0,18.11.15,"Lidl, Drayton Road, Norwich",NaN,NaN,Packaging:,Loose,Chilled,Chilled
145,7.0,"Cherries, raw",7.8,The Co -operative Loved by Us cherries,The Co-operative Loved by Us,7.6,12.08.15,"The Co -operative, Unthank Road, Norwich",Best before Date:,NaN,Supplied by: M150/TS P6729F/2/2,Plastic,Chilled,Chilled
147,7.0,"Cherries, raw",7.9,Sainsbury’s cherries,Sainsbury’s,9.0,16.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,A0800 X2,Plastic,Chilled,Chilled
149,7.0,"Cherries, raw",7.1,Market cherries,Pack size:,10.7,12.08.15,"Independent market stall, Norwich M arket",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled


Rename and Export

In [67]:
cherry.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
cherry.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\cherries.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\1997108774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cherry.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


##### Sample 8: Grapefruit, raw, flesh only
Grapefruit skin was removed either by peeling by hand and then removing the majority of the
pith or more usually, cutting the peel off. All pips were removed. Each sub-sample was taken
from eighths of three grapefruits.

##### Average weight of a grapefruit = 331.7g

In [68]:
Grape = saved_dataframes[10]
Grape

,Sub-sample\rnumber,Average\rweight of a\rwhole\rgrapefruit\r(g),Average\rweight of\rskin (g),Average\rweight of\rpips (g),Edible\rportion
0,8.10,357.0,126.7,0.7,0.64
1,8.20,330.3,95.1,0.3,0.71
2,8.30,324.9,109.9,0.4,0.66
3,8.40,326.7,74.8,1.2,0.77
4,8.50,291.6,69.6,0.8,0.76
5,8.60,318.5,92.4,0.3,0.71
6,8.70,331.9,118.7,0.0,0.64
7,8.80,338.9,122.5,0.1,0.64
8,8.90,329.3,76.7,1.4,0.76
9,8.10,313.7,79.4,1.1,0.74


In [69]:
Grape['Sub-sample\rnumber'] = Grape['Sub-sample\rnumber'].astype(str)

Doh text pdf 

In [70]:
grapefru = filtered_products[filtered_products['Composite Sample Name'].str.contains('Grapefruit')]
grapefru

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
153,8.0,"Grapefruit, raw, flesh only",8.1,Tesco pink grapefruit,Tesco,Purchase Date:,05.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
155,8.0,"Grapefruit, raw, flesh only",8.2,Tesco red grapefruit,Tesco,Purchase Date:,30.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
157,8.0,"Grapefruit, raw, flesh only",8.3,Asda Grower’s Selection red grapefruit,Asda Grower’s Selection,3 fruit,30.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: 791636 site: 901083 ASP1420022 SL1257,Netting,Ambient,Chilled
159,8.0,"Grapefruit, raw, flesh only",8.4,Sainsbury’s red grapefruit,Sainsbury’s,4 fruit,30.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,CO163A 05:01 0000 13,Plastic and netting,Ambient,Chilled
161,8.0,"Grapefruit, raw, flesh only",8.5,Morrisons pink grapefruit,Morrisons,Count 4,30.11.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,225033 TH1 CIT 2A 332 08:15 5G LA53700219,Plastic and netting,Ambient,21ºC controlled temperature room
163,8.0,"Grapefruit, raw, flesh only",8.6,Waitrose red grapefruit,Waitrose,Purchase Date:,30.11.15,"Waitrose, Wymondham",Best before Date:,NaN,4288,Loose,Ambient,21ºC controlled temperature room
165,8.0,"Grapefruit, raw, flesh only",8.7,Aldi red grapefruit,Aldi,Purchase Date:,05.10.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
167,8.0,"Grapefruit, raw, flesh only",8.8,Asda Grower’s Selection red grapefruit,Asda Grower’s Selection,3 fruit,05.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: 778546 Site: 902887 ASP1420022 SL1255,Plastic and netting,Ambient,21ºC controlled temperature room
169,8.0,"Grapefruit, raw, flesh only",8.9,The Shop SU white grapefruit,The Shop SU,Purchase Date:,22.09.15,"The Shop SU, University of East Anglia, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
171,8.0,"Grapefruit, raw, flesh only",8.1,Farm shop white grapefruit,Pack size:,Purchase Date:,13.10.15,"Independent farm shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room


In [71]:
grapefru['Sub-sample Number'] = grapefru['Sub-sample Number'].astype(str)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\2592491164.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grapefru['Sub-sample Number'] = grapefru['Sub-sample Number'].astype(str)


Match

In [72]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawgrapdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = Grape[Grape['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawgrapdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole\rgrapefruit\r(g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
grapefru['Pack size'] = grapefru.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\746806058.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grapefru['Pack size'] = grapefru.apply(update_pack_size, axis=1)


In [73]:
grapefru.tail()

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
165,8.0,"Grapefruit, raw, flesh only",8.7,Aldi red grapefruit,Aldi,331.9,05.10.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
167,8.0,"Grapefruit, raw, flesh only",8.8,Asda Grower’s Selection red grapefruit,Asda Grower’s Selection,338.9,05.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: 778546 Site: 902887 ASP1420022 SL1255,Plastic and netting,Ambient,21ºC controlled temperature room
169,8.0,"Grapefruit, raw, flesh only",8.9,The Shop SU white grapefruit,The Shop SU,329.3,22.09.15,"The Shop SU, University of East Anglia, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
171,8.0,"Grapefruit, raw, flesh only",8.1,Farm shop white grapefruit,Pack size:,357.0,13.10.15,"Independent farm shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,21°C controlled temperature room
173,8.0,"Grapefruit, raw, flesh only",8.11,Swithenbank red grapefruit,Swithenbank,385.7,12.10.15,Bidvest Foodservice delivery,Best before Date:,NaN,Trace code: H 104407 product code: 74939 #4288,Plastic,Ambient,21ºC controlled temperature room


In [74]:
grapefru.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
grapefru.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Grapefruit.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\3412890826.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  grapefru.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


##### Sample 9: Leeks, fresh, raw
The root and dark green top, as well as the outer leaf if damaged, was removed from each
leek. Trimmed leeks were then rinsed in Analar water and dried. Each sub-sample was taken
from lengthways eighths of 3-4 leeks.

##### Average weight of a leek = 254.2g

Doh table pdf

In [75]:
leek = saved_dataframes[11]
leek

,Sub-sample\rnumber,Average\rweight of a\rwhole leek\r(g),Average\rweight of\rbottom (g),Average\rweight of\rtop (g),Average\rweight of\rdiscarded\router leaves\r(g),Edible\rportion
0,9.10,271.7,9.3,60.3,11.0,0.70
1,9.20,254.7,9.7,99.9,0.0,0.57
2,9.30,172.2,6.8,9.9,27.1,0.75
3,9.40,353.0,11.5,106.5,16.0,0.62
4,9.50,295.7,13.4,106.9,9.5,0.56
5,9.60,145.3,6.4,0.0,0.0,0.96
6,9.70,189.3,10.7,31.0,12.0,0.72
7,9.80,111.4,20.7,5.4,0.0,0.77
8,9.90,146.2,5.1,42.6,7.0,0.63
9,9.10,378.2,7.1,147.4,40.5,0.48


In [76]:
leek['Sub-sample\rnumber'] = leek['Sub-sample\rnumber'].astype(str)

In [77]:
leek

,Sub-sample\rnumber,Average\rweight of a\rwhole leek\r(g),Average\rweight of\rbottom (g),Average\rweight of\rtop (g),Average\rweight of\rdiscarded\router leaves\r(g),Edible\rportion
0,9.1,271.7,9.3,60.3,11.0,0.70
1,9.2,254.7,9.7,99.9,0.0,0.57
2,9.3,172.2,6.8,9.9,27.1,0.75
3,9.4,353.0,11.5,106.5,16.0,0.62
4,9.5,295.7,13.4,106.9,9.5,0.56
5,9.6,145.3,6.4,0.0,0.0,0.96
6,9.7,189.3,10.7,31.0,12.0,0.72
7,9.8,111.4,20.7,5.4,0.0,0.77
8,9.9,146.2,5.1,42.6,7.0,0.63
9,9.1,378.2,7.1,147.4,40.5,0.48


Doh text pdf

In [89]:
leeks = filtered_products[filtered_products['Composite Sample Name'].str.contains('Leek')][:11]
leeks

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
175,9.0,"Leeks, fresh, raw",9.1,Tesco leeks,Tesco,Purchase Date:,19.10.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
177,9.0,"Leeks, fresh, raw",9.2,Tesco leeks,Tesco,500g,19.10.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC: EM12 L3288 10:46,Plastic,Chilled,Chilled
179,NaN,"Leeks, fresh, raw",9.3,Asda Grower’s Selection British extra trimmed ...,Asda Grower’s Selection,500g,16.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:143434 ASP1506594,Plastic,Chilled,Chilled
181,9.0,"Leeks, fresh, raw",9.4,Sainsbury’s British trimmed leeks,Sainsbury’s,500g,19.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,D288A 6 290 10:53 RO466 6308660 09.14,Plastic,Chilled,Chilled
183,9.0,"Leeks, fresh, raw",9.5,Morrisons trimmed leeks,Morrisons,500g,19.10.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,6999GSH 0522 T47518/0514,Plastic,Chilled,Chilled
185,9.0,"Leeks, fresh, raw",9.6,Sainsbury’s leeks,Sainsbury’s,Purchase Date:,19.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
187,9.0,"Leeks, fresh, raw",9.7,M&S British leeks,Marks & Spencer,600g,19.10.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,4146T,Plastic,Chilled,Chilled
189,9.0,"Leeks, fresh, raw",9.8,Oaklands British leeks,Oaklands,750g,19.10.15,NaN,Best before Date:,NaN,4204 E52144 W3552,Plastic,Chilled,Chilled
191,9.0,"Leeks, fresh, raw",9.9,The Co -operative leeks,The Co -operative,Purchase Date:,16.11.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
193,9.0,"Leeks, fresh, raw",9.1,Farm shop Leeks,Pack size:,Purchase Date:,13.10.15,"Independent farm shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled


In [79]:
leeks['Sub-sample Number'] = leeks['Sub-sample Number'].astype(str)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\2729702396.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  leeks['Sub-sample Number'] = leeks['Sub-sample Number'].astype(str)


Match

In [90]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawleekdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = leek[leek['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawleekdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole leek\r(g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
leeks['Pack size'] = leeks.apply(update_pack_size, axis=1)

In [92]:
leeks.tail()

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
187,9.0,"Leeks, fresh, raw",9.7,M&S British leeks,Marks & Spencer,189.3,19.10.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,4146T,Plastic,Chilled,Chilled
189,9.0,"Leeks, fresh, raw",9.8,Oaklands British leeks,Oaklands,111.4,19.10.15,NaN,Best before Date:,NaN,4204 E52144 W3552,Plastic,Chilled,Chilled
191,9.0,"Leeks, fresh, raw",9.9,The Co -operative leeks,The Co -operative,146.2,16.11.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
193,9.0,"Leeks, fresh, raw",9.1,Farm shop Leeks,Pack size:,271.7,13.10.15,"Independent farm shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
195,9.0,"Leeks, fresh, raw",9.11,Bidvest Foodservice Leeks,Bidvest Foodservice,478.7,12.10.15,Bidvest Foodservice delivery,Best before Date:,NaN,Packaging:,Plastic netting,Ambient,Chilled


In [93]:
leeks.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
leeks.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Leek.csv")

##### Sample 12: Courgettes, fresh, raw
Courgettes were rinsed in Analar water, dried and the end(s) removed. Each sub-sample was
taken from lengthways quarters of four courgettes.
##### Average weight of a courgette = 224.8g
Doh tables pdf

In [96]:
Courget1 = saved_dataframes[15]
Courget1

,Sub-sample\rnumber,Average\rweight of a\rwhole\rcourgette\r(g),Average\rweight of\rends (g),Edible\rportion
0,12.1,179.4,12.1,0.93


In [103]:
Courget1['Sub-sample\rnumber'] = Courget1['Sub-sample\rnumber'].astype(str)

In [101]:
Courget2 = saved_dataframes[16]

In [100]:
Courget2.columns = Courget1.columns

In [107]:
Courget2.head()

,Sub-sample\rnumber,Average\rweight of a\rwhole\rcourgette\r(g),Average\rweight of\rends (g),Edible\rportion
0,12.3,214.4,15.6,0.93
1,12.4,234.1,22.4,0.90
2,12.5,320.9,20.2,0.94
3,12.6,177.9,11.7,0.93
4,12.7,168.3,13.2,0.92


In [104]:
Courget2['Sub-sample\rnumber'] = Courget2['Sub-sample\rnumber'].astype(str)

In [108]:
Courget = pd.concat([Courget1,Courget2],ignore_index=True)
Courget.tail()

,Sub-sample\rnumber,Average\rweight of a\rwhole\rcourgette\r(g),Average\rweight of\rends (g),Edible\rportion
5,12.7,168.3,13.2,0.92
6,12.8,225.1,18.6,0.92
7,12.9,185.4,18.5,0.90
8,12.1,249.3,23.9,0.90
9,12.11,193.3,15.8,0.92


Doh text pdf

In [99]:
courgettes = filtered_products[filtered_products['Composite Sample Name'].str.contains('Courgettes')][:11]
courgettes

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
241,12.0,"Courgettes, fresh, raw",12.1,Tesco courgettes,Tesco,Purchase Date:,16.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
243,12.0,"Courgettes, fresh, raw",12.2,Tesco courgettes,Tesco,500g,16.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:GZ,Plastic,Chilled,Chilled
245,12.0,"Courgettes, fresh, raw",12.3,Asda Grower’s Selection British courgettes,Asda Grower’s Selection,3 pack,17.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: 12670 site: 950001 SS2 3 15:55 ASP14201...,Plastic,Chilled,Chilled
247,12.0,"Courgettes, fresh, raw",12.4,Sainsbury’s courgettes,Sainsbury’s,Purchase Date:,17.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
249,12.0,"Courgettes, fresh, raw",12.5,Morrisons courgettes,Morrisons,Purchase Date:,16.11.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
251,12.0,"Courgettes, fresh, raw",12.6,M&S courgettes,Marks & Spencer,325g,16.11.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC4146,Plastic,Chilled,Chilled
253,12.0,"Courgettes, fresh, raw",12.7,Wood Farm courgettes,Wood Farm,500g,17.08.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,3403 (3405C) Supplier code: 02985 25341195,Plastic,Chilled,Chilled
255,12.0,"Courgettes, fresh, raw",nan,Sainsbury’s Basics courgettes,Sainsbury’s Basics,1kg,17.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,6287853 09.13 M0249A,Plastic netting,Ambient,Chilled
257,12.0,"Courgettes, fresh, raw",12.9,Tesco British courgettes,Tesco,300g,19.08.15,"Tesco Express, Westlegate, Norwich",Best before Date:,NaN,SC:DV Oak -C 1,Plastic,Chilled,Chilled
259,12.0,"Courgettes, fresh, raw",12.1,Greengrocers courgettes,Pack size:,Purchase Date:,26.11.15,"Independent greengrocers, Norwich",Best before Date:,NaN,NaN,Loose,Ambient,Chilled


In [105]:
courgettes['Sub-sample Number'] = courgettes['Sub-sample Number'].astype(str)

Match

In [109]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawcougdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = Courget[Courget['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawcougdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole\rcourgette\r(g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
courgettes['Pack size'] = courgettes.apply(update_pack_size, axis=1)

In [110]:
courgettes

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
241,12.0,"Courgettes, fresh, raw",12.1,Tesco courgettes,Tesco,179.4,16.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
243,12.0,"Courgettes, fresh, raw",12.2,Tesco courgettes,Tesco,500g,16.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:GZ,Plastic,Chilled,Chilled
245,12.0,"Courgettes, fresh, raw",12.3,Asda Grower’s Selection British courgettes,Asda Grower’s Selection,214.4,17.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch: 12670 site: 950001 SS2 3 15:55 ASP14201...,Plastic,Chilled,Chilled
247,12.0,"Courgettes, fresh, raw",12.4,Sainsbury’s courgettes,Sainsbury’s,234.1,17.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
249,12.0,"Courgettes, fresh, raw",12.5,Morrisons courgettes,Morrisons,320.9,16.11.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
251,12.0,"Courgettes, fresh, raw",12.6,M&S courgettes,Marks & Spencer,177.9,16.11.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC4146,Plastic,Chilled,Chilled
253,12.0,"Courgettes, fresh, raw",12.7,Wood Farm courgettes,Wood Farm,168.3,17.08.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,3403 (3405C) Supplier code: 02985 25341195,Plastic,Chilled,Chilled
255,12.0,"Courgettes, fresh, raw",nan,Sainsbury’s Basics courgettes,Sainsbury’s Basics,1kg,17.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,6287853 09.13 M0249A,Plastic netting,Ambient,Chilled
257,12.0,"Courgettes, fresh, raw",12.9,Tesco British courgettes,Tesco,185.4,19.08.15,"Tesco Express, Westlegate, Norwich",Best before Date:,NaN,SC:DV Oak -C 1,Plastic,Chilled,Chilled
259,12.0,"Courgettes, fresh, raw",12.1,Greengrocers courgettes,Pack size:,179.4,26.11.15,"Independent greengrocers, Norwich",Best before Date:,NaN,NaN,Loose,Ambient,Chilled


In [111]:
courgettes.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
courgettes.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Courgettes.csv")

##### Sample 15: Brussels sprouts, boiled
The bottom and any outer leaves were removed from each Brussels sprout. A cross was cut
into the bottom before rinsing, boiling for 4-10 minutes and draining. Each sub-sample was
taken from about 20 halves or whole boiled sprouts.
##### Average weight of a whole sprout = 17.0g

In [113]:
sprout = saved_dataframes[20]
sprout

,Sub-sample\rnumber,Average\rweight of a\rwhole\rsprout (g),Average\rweight of\rends (g),Average\rweight of\rremoved\rleaves (g),Edible\rportion
0,15.10,29.4,2.5,4.5,0.76
1,15.20,14.7,N/A,N/A,N/A
2,15.30,15.6,1.4,1.3,0.83
3,15.40,16.0,1.1,2.4,0.78
4,15.50,22.7,1.5,4.3,0.74
5,15.60,6.1,0.4,0.7,0.82
6,15.70,10.1,1.1,1.1,0.79
7,15.80,20.9,2.4,1.9,0.80
8,15.90,9.4,1.3,0.6,0.80
9,15.10,21.8,2.0,3.8,0.74


In [115]:
sprout['Sub-sample\rnumber'] = sprout['Sub-sample\rnumber'].astype(str)

In [114]:
brussel = filtered_products[filtered_products['Composite Sample Name'].str.contains('Brussels')][:11]
brussel

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
307,15.0,"Brussels sprouts, boiled",15.1,Tesco Brussels sprouts,Tesco,Purchase Date:,19.10.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
309,15.0,"Brussels sprouts, boiled",15.2,Tesco British peeled Brussels sprouts,Tesco,250g,19.10.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC:DR,Plastic,Chilled,Chilled
311,15.0,"Brussels sprouts, boiled",15.3,Asda Grower’s Selection British sprouts,Asda Grower’s Selection,500g,23.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,SPRT 289 19 37 A 00362 AP1508557,Plastic,NaN,Chilled
313,15.0,"Brussels sprouts, boiled",15.4,Sainsbury’s Brussels sprouts,Sainsbury’s,Purchase Date:,19.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,Packaging:,Loose,NaN,Chilled
315,15.0,"Brussels sprouts, boiled",15.5,Morrisons Brussels sprouts,Morrisons,500g,19.10.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1825/WRA 2725 FL53705594 T52691/1214,Plastic,Chilled,Chilled
317,15.0,"Brussels sprouts, boiled",15.6,M&S British baby sprouts,Marks & Spencer,200g,19.10.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC4146W,Plastic,Chilled,Chilled
319,15.0,"Brussels sprouts, boiled",15.7,Wood Farm Brussels sprouts,Wood Farm,500g,19.10.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,L 4301 (4303c) S939 CEE ALR14073S3 25341201,Plastic,Chilled,Chilled
321,15.0,"Brussels sprouts, boiled",15.8,Tesco British Brussels sprouts,Tesco,500g,26.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:DR,Plastic,Chilled,Chilled
323,15.0,"Brussels sprouts, boiled",15.9,The Co -operative Loved by Us British Brussels...,The Co -operative Loved by Us,500g,26.10.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,M233 SPRT 297 18:37 B 01213,Plastic,Chilled,Chilled
325,NaN,"Brussels sprouts, boiled",15.1,Farm Shop Brussels sprouts,Pack size:,Purchase Date:,13.10.15,"Independent Farm Shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled


In [116]:
brussel['Sub-sample Number'] = brussel['Sub-sample Number'].astype(str)

Match

In [117]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawsprotdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = sprout[sprout['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawsprotdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole\rsprout (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
brussel['Pack size'] = brussel.apply(update_pack_size, axis=1)

In [118]:
brussel

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
307,15.0,"Brussels sprouts, boiled",15.1,Tesco Brussels sprouts,Tesco,29.4,19.10.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
309,15.0,"Brussels sprouts, boiled",15.2,Tesco British peeled Brussels sprouts,Tesco,14.7,19.10.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC:DR,Plastic,Chilled,Chilled
311,15.0,"Brussels sprouts, boiled",15.3,Asda Grower’s Selection British sprouts,Asda Grower’s Selection,15.6,23.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,SPRT 289 19 37 A 00362 AP1508557,Plastic,NaN,Chilled
313,15.0,"Brussels sprouts, boiled",15.4,Sainsbury’s Brussels sprouts,Sainsbury’s,16.0,19.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,Packaging:,Loose,NaN,Chilled
315,15.0,"Brussels sprouts, boiled",15.5,Morrisons Brussels sprouts,Morrisons,22.7,19.10.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1825/WRA 2725 FL53705594 T52691/1214,Plastic,Chilled,Chilled
317,15.0,"Brussels sprouts, boiled",15.6,M&S British baby sprouts,Marks & Spencer,6.1,19.10.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC4146W,Plastic,Chilled,Chilled
319,15.0,"Brussels sprouts, boiled",15.7,Wood Farm Brussels sprouts,Wood Farm,10.1,19.10.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,L 4301 (4303c) S939 CEE ALR14073S3 25341201,Plastic,Chilled,Chilled
321,15.0,"Brussels sprouts, boiled",15.8,Tesco British Brussels sprouts,Tesco,20.9,26.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:DR,Plastic,Chilled,Chilled
323,15.0,"Brussels sprouts, boiled",15.9,The Co -operative Loved by Us British Brussels...,The Co -operative Loved by Us,9.4,26.10.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,M233 SPRT 297 18:37 B 01213,Plastic,Chilled,Chilled
325,NaN,"Brussels sprouts, boiled",15.1,Farm Shop Brussels sprouts,Pack size:,29.4,13.10.15,"Independent Farm Shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled


In [119]:
brussel.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
brussel.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Brusselsprout.csv")

##### Sample 16: Parsnips, boiled
Parsnips were rinsed in Analar water and dried before removing the top and peel. Trimmed
parsnips were cut into chunks or slices before boiling for 8-12 minutes and draining. Each subsample
was taken randomly from the boiled pieces from 2-3 parsnips.
##### Average weight of a parsnip = 145.4

In [120]:
pars = saved_dataframes[23]
pars

,Sub-sample\rnumber,Average\rweight of a\rwhole\rparsnip (g),Average\rweight of\rtop and\rbottom (g),Average\rweight of\rpeel (g),Edible\rportion
0,16.10,148.7,19.0,18.1,0.75
1,16.20,116.8,12.7,18.7,0.73
2,16.30,85.8,8.8,18.5,0.68
3,16.40,132.1,14.7,16.3,0.77
4,16.50,136.6,14.9,23.4,0.72
5,16.60,144.3,23.6,15.8,0.73
6,16.70,102.8,10.9,15.5,0.74
7,16.80,147.3,14.2,18.1,0.78
8,16.90,225.5,22.5,22.6,0.80
9,16.10,164.0,15.7,19.2,0.79


In [121]:
pars['Sub-sample\rnumber'] = pars['Sub-sample\rnumber'].astype(str)

In [123]:
parsnip = filtered_products[filtered_products['Composite Sample Name'].str.contains('Parsnips')][:11]
parsnip

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
329,16.0,"Parsnips, boiled",16.1,Tesco parsnips,Tesco,Purchase Date:,02.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
331,16.0,"Parsnips, boiled",16.2,Tesco British parsnips,Tesco,500g,02.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:ABC D7 08:55,Plastic,Ambient,Chilled
333,16.0,"Parsnips, boiled",16.3,Asda Grower’s Selection British parsnips,Asda Grower’s Selection,500g,02.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch:115 site:900322 ASP1508736,Plastic,Ambient,Chilled
335,16.0,"Parsnips, boiled",16.4,Sainsbury’s parsnips,Sainsbury’s,Purchase Date:,02.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
337,16.0,"Parsnips, boiled",16.5,Morrisons parsnips,Morrisons,500g,02.11.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,0796/HAM -303-15-08:48 -Etna 001 2725 T50866/0...,Plastic,Ambient,Chilled
339,16.0,"Parsnips, boiled",16.6,M&S parsnips,Marks & Spencer,Purchase Date:,02.11.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
341,16.0,"Parsnips, boiled",16.7,Oaklands British parsnips,Oaklands,500g,09.11.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,4504 09:45 W1133 6039 6A,Plastic,Ambient,Chilled
343,16.0,"Parsnips, boiled",16.8,Sainsbury’s Basics parsnips,Sainsbury’s Basics,750g,02.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,I0062A 302 07:176B 7169946 10:13,Plastic,Ambient,Chilled
345,16.0,"Parsnips, boiled",16.9,The Co -operative parsnips,The Co -operative,Purchase Date:,03.11.15,"The Co -operative, Wymondham",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
347,16.0,"Parsnips, boiled",16.1,Greengrocer parsnips,Pack size:,Purchase Date:,03.11.15,"Independent greengrocer, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled


In [124]:
parsnip['Sub-sample Number'] = parsnip['Sub-sample Number'].astype(str)

Match

In [125]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawsprotdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = pars[pars['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawsprotdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole\rparsnip (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
parsnip['Pack size'] = parsnip.apply(update_pack_size, axis=1)

In [126]:
parsnip

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
329,16.0,"Parsnips, boiled",16.1,Tesco parsnips,Tesco,148.7,02.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
331,16.0,"Parsnips, boiled",16.2,Tesco British parsnips,Tesco,116.8,02.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:ABC D7 08:55,Plastic,Ambient,Chilled
333,16.0,"Parsnips, boiled",16.3,Asda Grower’s Selection British parsnips,Asda Grower’s Selection,85.8,02.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Batch:115 site:900322 ASP1508736,Plastic,Ambient,Chilled
335,16.0,"Parsnips, boiled",16.4,Sainsbury’s parsnips,Sainsbury’s,132.1,02.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
337,16.0,"Parsnips, boiled",16.5,Morrisons parsnips,Morrisons,136.6,02.11.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,0796/HAM -303-15-08:48 -Etna 001 2725 T50866/0...,Plastic,Ambient,Chilled
339,16.0,"Parsnips, boiled",16.6,M&S parsnips,Marks & Spencer,144.3,02.11.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
341,16.0,"Parsnips, boiled",16.7,Oaklands British parsnips,Oaklands,102.8,09.11.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,4504 09:45 W1133 6039 6A,Plastic,Ambient,Chilled
343,16.0,"Parsnips, boiled",16.8,Sainsbury’s Basics parsnips,Sainsbury’s Basics,147.3,02.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,I0062A 302 07:176B 7169946 10:13,Plastic,Ambient,Chilled
345,16.0,"Parsnips, boiled",16.9,The Co -operative parsnips,The Co -operative,225.5,03.11.15,"The Co -operative, Wymondham",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
347,16.0,"Parsnips, boiled",16.1,Greengrocer parsnips,Pack size:,148.7,03.11.15,"Independent greengrocer, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled


In [127]:
parsnip.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
parsnip.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\parsnip.csv")

##### Sample 18: Beetroot, cooked
The top and root was removed from raw beetroot samples before boiling for 1-2 hours. The skin
removed after boiling. Each sub-sample was taken from quarters of 3-5 cooked beetroot.
##### Average weight of a raw beetroot with leaves = 134.9g

In [130]:
beet = saved_dataframes[27]
beet

,Sub-sample\rnumber,Average\rweight of a\rwhole raw\rbeetroot (g),Average\rweight of\rraw ends\r(g),Average\rweight of\rraw leaves\r(g),Edible\rportion
0,18.3,116.8,16.3,1.4,0.85
1,18.4,133.2,25.6,0.7,0.80


In [134]:
beet1 = saved_dataframes[28]

In [132]:
beet1.columns = beet.columns

In [135]:
beetr = pd.concat([beet,beet1], ignore_index=True)

In [136]:
beetr

,Sub-sample\rnumber,Average\rweight of a\rwhole raw\rbeetroot (g),Average\rweight of\rraw ends\r(g),Average\rweight of\rraw leaves\r(g),Edible\rportion
0,18.3,116.8,16.3,1.4,0.85
1,18.4,133.2,25.6,0.7,0.80
2,18.1,181.8,72.2,1.0,0.60


In [140]:
beetr['Sub-sample\rnumber'] = beetr['Sub-sample\rnumber'].astype(str)

In [138]:
beetroot = filtered_products[filtered_products['Composite Sample Name'].str.contains('Beetroot')][:4]
beetroot

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
373,18.0,"Beetroot, cooked",18.1,Tesco cooked British beetroot,Tesco,300g,28.09.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,5222 A10,Plastic,Chilled,Chilled
375,18.0,"Beetroot, cooked",18.2,Tesco cooked British beetroot,Tesco,300g,05.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,5229 L8,Plastic,Ambient,Chilled
377,18.0,"Beetroot, cooked",18.3,Asda Grower’s Selection bunched beetroot,Asda Grower’s Selection,500g,28.09.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,ASP1507042 site:700292,Loose; tag: mixed material,Ambient,Chilled
379,18.0,"Beetroot, cooked",18.4,Sainsbury’s bunched beetroot,Sainsbury’s,500g,28.09.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,6031981 10062C 11.14,Loose,Ambient,Chilled


In [141]:
beetroot['Sub-sample Number'] = beetroot['Sub-sample Number'].astype(str)

Match

In [142]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawbeetdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = beetr[beetr['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawbeetdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole raw\rbeetroot (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
beetroot['Pack size'] = beetroot.apply(update_pack_size, axis=1)

In [143]:
beetroot

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
373,18.0,"Beetroot, cooked",18.1,Tesco cooked British beetroot,Tesco,181.8,28.09.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,5222 A10,Plastic,Chilled,Chilled
375,18.0,"Beetroot, cooked",18.2,Tesco cooked British beetroot,Tesco,300g,05.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,5229 L8,Plastic,Ambient,Chilled
377,18.0,"Beetroot, cooked",18.3,Asda Grower’s Selection bunched beetroot,Asda Grower’s Selection,116.8,28.09.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,ASP1507042 site:700292,Loose; tag: mixed material,Ambient,Chilled
379,18.0,"Beetroot, cooked",18.4,Sainsbury’s bunched beetroot,Sainsbury’s,133.2,28.09.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,6031981 10062C 11.14,Loose,Ambient,Chilled


In [144]:
beetroot.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
beetroot.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Beetroot.csv")

##### Sample 19: Sweetcorn, baby, fresh, boiled
Baby sweetcorn were rinsed in Analar water before boiling for 4-13 minutes and draining. Each
sub-sample consisted of 10-15 whole baby sweetcorn.

In [147]:
sweet = saved_dataframes[31]
sweet

,Sub-\rsample\rnumber,Boiling\rtime\r(minutes),Weight\rbefore\rcooking (g),Weight\rafter\rcooking (g),Weight\rchange\r(%)
0,19.10,5,401.7,381.9,-4.9
1,19.20,8,414.9,397.3,-4.2
2,19.30,7,379.5,364.5,-4.0
3,19.40,4,340.8,334.9,-1.7
4,19.50,6,405.5,392.1,-3.3
5,19.60,7,374.8,351.5,-6.2
6,19.70,5,360.0,353.0,-1.9
7,19.80,6,375.0,368.0,-1.9
8,19.90,13,415.9,400.0,-3.8
9,19.10,8,367.2,355.3,-3.2


In [148]:
sweet['Sub-\rsample\rnumber'] = sweet['Sub-\rsample\rnumber'].astype(str)

In [150]:
corn = filtered_products[filtered_products['Composite Sample Name'].str.contains('Sweetcorn')][:11]
corn

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
395,19.0,"Sweetcorn, baby, fresh, boiled",19.1,Tesco baby corn,Tesco,190g,09.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HM2fd,Plastic,Chilled,Chilled
397,19.0,"Sweetcorn, baby, fresh, boiled",19.2,Tesco baby corn,Tesco,190g,09.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC HM28,Plastic,Chilled,Chilled
399,19.0,"Sweetcorn, baby, fresh, boiled",19.3,Asda Grower’s Selection baby corn,Asda Grower’s Selection,130g,09.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:550489 ASP1419792,Plastic,Chilled,Chilled
401,19.0,"Sweetcorn, baby, fresh, boiled",19.4,Sainsbury’s babycorn,Sainsbury’s,175g,30.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,P0698G 1043433 12.13,Plastic,Chilled,Chilled
403,19.0,"Sweetcorn, baby, fresh, boiled",19.5,Morrisons baby corn,Morrisons,125g,09.11.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,T360/MWW T48399/0614,Plastic,Chilled,Chilled
405,19.0,"Sweetcorn, baby, fresh, boiled",19.6,Waitrose baby corn,Waitrose,175g,18.11.15,"Waitrose, Wymondham , Norfolk",23.11.15,NaN,Supplier 284GND 10.14,Plastic,Chilled,Chilled
407,19.0,"Sweetcorn, baby, fresh, boiled",19.7,Wood Farm baby corn,Wood Farm,135g,16.11.15,"Aldi, Sprowston Road, Norwich",Best before Date:,NaN,Supplier code:03107/HQC Lot code:4606 (4701C),Plastic,Chilled,Chilled
409,19.0,"Sweetcorn, baby, fresh, boiled",19.8,Asda Grower’s Selection baby corn,Asda Grower’s Selection,130g,30.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site: 550515 ASP1419792,Plastic,Chilled,Chilled
411,19.0,"Sweetcorn, baby, fresh, boiled",19.9,The Co -operative Loved by Us babycorn,The Co -operative Loved by Us,125g,09.11.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Supplied by: M262B P6470/1/3 5000128471152,Plastic,Chilled,Chilled
413,19.0,"Sweetcorn, baby, fresh, boiled",19.1,Morrisons baby corn,Morrisons,125g,16.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,T360/MWW T48399/0614,Plastic,Chilled,Chilled


In [151]:
corn['Sub-sample Number'] = corn['Sub-sample Number'].astype(str)

Match

In [153]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawsweetdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = sweet[sweet['Sub-\rsample\rnumber'].apply(lambda x: regex_matching(rawsweetdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Weight\rbefore\rcooking (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
corn['Pack size'] = corn.apply(update_pack_size, axis=1)

In [154]:
corn

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
395,19.0,"Sweetcorn, baby, fresh, boiled",19.1,Tesco baby corn,Tesco,401.7,09.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HM2fd,Plastic,Chilled,Chilled
397,19.0,"Sweetcorn, baby, fresh, boiled",19.2,Tesco baby corn,Tesco,414.9,09.11.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC HM28,Plastic,Chilled,Chilled
399,19.0,"Sweetcorn, baby, fresh, boiled",19.3,Asda Grower’s Selection baby corn,Asda Grower’s Selection,379.5,09.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:550489 ASP1419792,Plastic,Chilled,Chilled
401,19.0,"Sweetcorn, baby, fresh, boiled",19.4,Sainsbury’s babycorn,Sainsbury’s,340.8,30.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,P0698G 1043433 12.13,Plastic,Chilled,Chilled
403,19.0,"Sweetcorn, baby, fresh, boiled",19.5,Morrisons baby corn,Morrisons,405.5,09.11.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,T360/MWW T48399/0614,Plastic,Chilled,Chilled
405,19.0,"Sweetcorn, baby, fresh, boiled",19.6,Waitrose baby corn,Waitrose,374.8,18.11.15,"Waitrose, Wymondham , Norfolk",23.11.15,NaN,Supplier 284GND 10.14,Plastic,Chilled,Chilled
407,19.0,"Sweetcorn, baby, fresh, boiled",19.7,Wood Farm baby corn,Wood Farm,360.0,16.11.15,"Aldi, Sprowston Road, Norwich",Best before Date:,NaN,Supplier code:03107/HQC Lot code:4606 (4701C),Plastic,Chilled,Chilled
409,19.0,"Sweetcorn, baby, fresh, boiled",19.8,Asda Grower’s Selection baby corn,Asda Grower’s Selection,375.0,30.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site: 550515 ASP1419792,Plastic,Chilled,Chilled
411,19.0,"Sweetcorn, baby, fresh, boiled",19.9,The Co -operative Loved by Us babycorn,The Co -operative Loved by Us,415.9,09.11.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Supplied by: M262B P6470/1/3 5000128471152,Plastic,Chilled,Chilled
413,19.0,"Sweetcorn, baby, fresh, boiled",19.1,Morrisons baby corn,Morrisons,401.7,16.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,T360/MWW T48399/0614,Plastic,Chilled,Chilled


In [155]:
corn.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
corn.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Sweetcorn.csv")

##### Sample 21: Celery, raw
The bottom and leaves were removed from celery bunches. Trimmed sticks for each subsample
were then rinsed in Analar water and dried. Each sub-sample was taken from
lengthways strips or half stems from 8-10 inner and outer stems.

##### Average celery bunch = 515.1g

In [156]:
cel = saved_dataframes[33]
cel

,Sub-sample\rnumber,Average\rweight of a\rcelery\rbunch (g),Edible parts\rof a bunch\r(g),Average\rwhole stem\rwithout\rleaves (g),Edible\rportion of a\rbunch
0,21.10,N/A,N/A,34.8,N/A
1,21.20,N/A,N/A,42.9,N/A
2,21.30,216.7,166.9,23.9,0.23
3,21.40,611.6,441.7,38.9,0.28
4,21.50,679.7,500.8,49.9,0.26
5,21.60,506.4,385.7,41.6,0.24
6,21.70,528.2,421.7,33.7,0.20
7,21.80,N/A,N/A,47.6,N/A
8,21.90,598.9,456.9,54.8,0.24
9,21.10,531.3,416.6,39.9,0.22


In [157]:
cel['Sub-sample\rnumber'] = cel['Sub-sample\rnumber'].astype(str)

In [158]:
celery = filtered_products[filtered_products['Composite Sample Name'].str.contains('Celery')][:11]
celery

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
439,21.0,"Celery, raw",21.1,Tesco British celery sticks,Tesco,350g,26.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:GS01,Plastic,Chilled,Chilled
441,21.0,"Celery, raw",21.2,Tesco celery sticks,Tesco,350g,16.11.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC:GS02,Plastic,Chilled,Chilled
443,21.0,"Celery, raw",21.3,Asda Grower’s Selection British celery hearts,Asda Grower’s Selection,Purchase Date:,26.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:470021 ASP1418547,Plastic,Chilled,Chilled
445,21.0,"Celery, raw",21.4,Sainsbury’s celery,Sainsbury’s,Purchase Date:,09.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,G0630F 805353 09.14,Plastic,Chilled,Chilled
447,21.0,"Celery, raw",21.5,Morrisons celery,Morrisons,Purchase Date:,26.08.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,6999BBT T50853/0415 2725,Plastic,Chilled,Chilled
449,21.0,"Celery, raw",21.6,Simply M& S British celery,Simply M&S,Purchase Date:,26.08.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC1816,Plastic,Chilled,Chilled
451,21.0,"Celery, raw",21.7,Oaklands British celery,Oaklands,Purchase Date:,26.08.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,W1922 E52257 3501,Plastic,Chilled,Chilled
453,21.0,"Celery, raw",21.8,Tesco celery sticks,Tesco,350g,16.11.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC:GS02,Plastic,Chilled,Chilled
455,21.0,"Celery, raw",21.9,The Co -operative Love by Us British celery,The Co -operative Love by Us,Purchase Date:,26.08.15,"The Co -operative, Earlham Green Lane, Norwich",Best before Date:,NaN,Supplied by: M029(L) P12574/1/2,Plastic,Chilled,Chilled
457,21.0,"Celery, raw",21.1,G’s celery,G’s,Purchase Date:,28.08.15,"Independent market stall, Norwich Market",Best before Date:,NaN,Packaging:,Plastic,Ambient,Chilled


In [159]:
celery['Sub-sample Number'] = celery['Sub-sample Number'].astype(str)

Match

In [161]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawceldf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = cel[cel['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawceldf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rcelery\rbunch (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
celery['Pack size'] = celery.apply(update_pack_size, axis=1)

In [162]:
celery

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
439,21.0,"Celery, raw",21.1,Tesco British celery sticks,Tesco,N/A,26.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:GS01,Plastic,Chilled,Chilled
441,21.0,"Celery, raw",21.2,Tesco celery sticks,Tesco,N/A,16.11.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC:GS02,Plastic,Chilled,Chilled
443,21.0,"Celery, raw",21.3,Asda Grower’s Selection British celery hearts,Asda Grower’s Selection,216.7,26.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:470021 ASP1418547,Plastic,Chilled,Chilled
445,21.0,"Celery, raw",21.4,Sainsbury’s celery,Sainsbury’s,611.6,09.11.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,G0630F 805353 09.14,Plastic,Chilled,Chilled
447,21.0,"Celery, raw",21.5,Morrisons celery,Morrisons,679.7,26.08.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,6999BBT T50853/0415 2725,Plastic,Chilled,Chilled
449,21.0,"Celery, raw",21.6,Simply M& S British celery,Simply M&S,506.4,26.08.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC1816,Plastic,Chilled,Chilled
451,21.0,"Celery, raw",21.7,Oaklands British celery,Oaklands,528.2,26.08.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,W1922 E52257 3501,Plastic,Chilled,Chilled
453,21.0,"Celery, raw",21.8,Tesco celery sticks,Tesco,N/A,16.11.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC:GS02,Plastic,Chilled,Chilled
455,21.0,"Celery, raw",21.9,The Co -operative Love by Us British celery,The Co -operative Love by Us,598.9,26.08.15,"The Co -operative, Earlham Green Lane, Norwich",Best before Date:,NaN,Supplied by: M029(L) P12574/1/2,Plastic,Chilled,Chilled
457,21.0,"Celery, raw",21.1,G’s celery,G’s,N/A,28.08.15,"Independent market stall, Norwich Market",Best before Date:,NaN,Packaging:,Plastic,Ambient,Chilled


In [164]:
celery.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
celery.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Celery.csv")

##### Sample 23: Asparagus, fresh, steamed
Asparagus was not rinsed. Experimentation showed that, when rinsed, the spear tips were
susceptible to damage. Spears were trimmed and extra-large ones cut in half before steaming
for 3-18 minutes. Each sub-sample was randomly taken from the steamed asparagus of 12-25
spears.
##### Average weight of an asparagus spear = 16.6g

In [166]:
asp = saved_dataframes[35]
asp

,Sub-sample\rnumber,Average\rweight of an\rasparagus\rbunch (g),Average\rweight of a\rspear (g),Average\rweight of\rthe end (g),Edible\rportion
0,23.10,254,28.8,3.3,0.89
1,23.20,N/A,10.3,N/A,N/A
2,23.30,272.0,16.3,2.0,0.88
3,23.40,121.5,8.4,1.3,0.85
4,23.50,283,27.0,6.0,0.78
5,23.60,245.3,18.9,2.2,0.83
6,23.70,107.2,6.3,1.3,0.80
7,23.80,N/A,9.8,1.8,0.82
8,23.90,N/A,8.0,N/A,N/A
9,23.10,285.9,20.9,8.0,0.62


In [170]:
asp['Sub-sample\rnumber'] = asp['Sub-sample\rnumber'].astype(str)

In [168]:
Asparagus = filtered_products[filtered_products['Composite Sample Name'].str.contains('Asparagus')][:10]
Asparagus

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
483,23.0,"Asparagus, fresh, steamed",23.1,Tesco asparagus bundle,Tesco,250g,26.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:WD02,Plastic,Chilled,Chilled
485,23.0,"Asparagus, fresh, steamed",23.2,Tesco asparagus tips,Tesco,125g,26.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HM2p,Plastic,Chilled,Chilled
487,23.0,"Asparagus, fresh, steamed",23.3,Asda Grower’s Selection asparagus,Asda Grower’s Selection,250g,26.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,550303 ASP1508538,Plastic,Chilled,Chilled
489,23.0,"Asparagus, fresh, steamed",23.4,Sainsbury’s fine asparagus,Sainsbury’s,100g,05.07.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,B0090L 7689821 03.14,Plastic,Chilled,Chilled
491,23.0,"Asparagus, fresh, steamed",23.5,Morrisons asparagus,Morrisons,250g,26.08.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,0589/2f 0589 T47641/0514,Plastic,Chilled,Chilled
495,23.0,"Asparagus, fresh, steamed",23.7,M&S extra fine asparagus,Marks & Spencer,110g,04.07.15,"Marks & Spencer, Rampant Horse Street, Norwich",Best before Date:,NaN,SC4848J,Plastic,Chilled,Chilled
497,23.0,"Asparagus, fresh, steamed",23.8,Aldi Specially Selected asparagus tips,Aldi Specially Selected,100g,26.08.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,Supplier code: 03107/SFT; lot code:3502(3504C),Plastic,Chilled,Chilled
499,23.0,"Asparagus, fresh, steamed",23.9,Tesco asparagus tips,Tesco,100g,28.08.15,"Tesco Express, Westlegate, Norwich",Best before Date:,NaN,SC:HM2p,Plastic,Chilled,Chilled
501,23.0,"Asparagus, fresh, steamed",23.1,Farm Shop asparagus,Pack size:,Purchase Date:,05.07.15,"Independent Farm Shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
503,23.0,"Asparagus, fresh, steamed",23.11,Bidvest Foodservice asparagus,Bidvest Foodservice,Purchase Date:,24.08.15,Bidvest Foodservice delivery,Best before Date:,NaN,PLU 4080,Loose,Chilled,Chilled


In [171]:
Asparagus['Sub-sample Number'] = Asparagus['Sub-sample Number'].astype(str)

Match

In [172]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawaspdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = asp[asp['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawaspdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of an\rasparagus\rbunch (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
Asparagus['Pack size'] = Asparagus.apply(update_pack_size, axis=1)

In [173]:
Asparagus

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
483,23.0,"Asparagus, fresh, steamed",23.1,Tesco asparagus bundle,Tesco,254,26.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:WD02,Plastic,Chilled,Chilled
485,23.0,"Asparagus, fresh, steamed",23.2,Tesco asparagus tips,Tesco,N/A,26.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HM2p,Plastic,Chilled,Chilled
487,23.0,"Asparagus, fresh, steamed",23.3,Asda Grower’s Selection asparagus,Asda Grower’s Selection,272.0,26.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,550303 ASP1508538,Plastic,Chilled,Chilled
489,23.0,"Asparagus, fresh, steamed",23.4,Sainsbury’s fine asparagus,Sainsbury’s,121.5,05.07.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,B0090L 7689821 03.14,Plastic,Chilled,Chilled
491,23.0,"Asparagus, fresh, steamed",23.5,Morrisons asparagus,Morrisons,283,26.08.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,0589/2f 0589 T47641/0514,Plastic,Chilled,Chilled
495,23.0,"Asparagus, fresh, steamed",23.7,M&S extra fine asparagus,Marks & Spencer,107.2,04.07.15,"Marks & Spencer, Rampant Horse Street, Norwich",Best before Date:,NaN,SC4848J,Plastic,Chilled,Chilled
497,23.0,"Asparagus, fresh, steamed",23.8,Aldi Specially Selected asparagus tips,Aldi Specially Selected,N/A,26.08.15,"Aldi, Larkman Lane, Norwich",Best before Date:,NaN,Supplier code: 03107/SFT; lot code:3502(3504C),Plastic,Chilled,Chilled
499,23.0,"Asparagus, fresh, steamed",23.9,Tesco asparagus tips,Tesco,N/A,28.08.15,"Tesco Express, Westlegate, Norwich",Best before Date:,NaN,SC:HM2p,Plastic,Chilled,Chilled
501,23.0,"Asparagus, fresh, steamed",23.1,Farm Shop asparagus,Pack size:,254,05.07.15,"Independent Farm Shop, Mulbarton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,Chilled
503,23.0,"Asparagus, fresh, steamed",23.11,Bidvest Foodservice asparagus,Bidvest Foodservice,363.6,24.08.15,Bidvest Foodservice delivery,Best before Date:,NaN,PLU 4080,Loose,Chilled,Chilled


In [174]:
Asparagus.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
Asparagus.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Asparagus.csv")

##### Sample 26: Mangetout, fresh, boiled
Mangetout were rinsed in Analar water, boiled for 3-6 minutes and drained. Each sub-sample
was randomly taken from 75-150 boiled mangetout.

In [180]:
Mange = saved_dataframes[39]
Mange

,Sub-\rsample\rnumber,Boiling\rtime\r(minutes),Weight\rbefore\rcooking\r(g),Weight\rafter\rcooking\r(g),Weight\rchange\r(%)
0,26.10,6,390.0,376.0,-3.6
1,26.20,5,319.1,310.4,-2.7
2,26.30,3,402.0,387.0,-3.7
3,26.40,3,417.0,401.0,-3.8
4,26.50,4,441.0,423.7,-3.9
5,26.60,5,360.8,354.1,-1.9
6,26.70,4,402.0,397.0,-1.2
7,26.80,3,341.4,319.8,-6.3
8,26.90,6,306.9,313.7,2.2
9,26.10,4,387.7,380.1,-2.0


In [181]:
Mange['Sub-\rsample\rnumber'] = Mange['Sub-\rsample\rnumber'].astype(str)

In [188]:
Mangetout = filtered_products[filtered_products['Composite Sample Name'].str.contains('Mange tout')][:11]
Mangetout

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
533,26.0,"Mange tout, fresh, boiled",26.1,Tesco mangetout,Tesco,215g,05.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HM2e,Plastic,Chilled,Chilled
535,26.0,"Mange tout, fresh, boiled",26.2,Sainsbury’s mange tout,Sainsbury’s,200g,07.12.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,P0698D 1039405,Plastic,Chilled,Chilled
537,26.0,"Mange tout, fresh, boiled",26.3,Asda Grower’s Selection mangetout,Asda Grower’s Selection,155g,05.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site: 550214 ASP1508372,Plastic,Chilled,Chilled
539,26.0,"Mange tout, fresh, boiled",26.4,Sainsbury’s mange tout,Sainsbury’s,200g,05.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,P0698D 1039405 09.14,Plastic,Chilled,Chilled
541,26.0,"Mange tout, fresh, boiled",26.5,Morrisons trimmed mangetout,Morrisons,155g,19.10.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1360/MWW,Plastic,Chilled,Chilled
543,26.0,"Mange tout, fresh, boiled",26.6,Waitrose trimmed mange tout peas,Waitrose,200g,18.11.15,"Waitrose, Wymondham , Norfolk",22.11.15,NaN,Supplier: 343SIS 11.14,Plastic,Chilled,Chilled
545,26.0,"Mange tout, fresh, boiled",26.7,Oaklands fresh mangetout,Oaklands,150g,05.10.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,GB005 –W4062 L4006 14:34,Plastic,Chilled,Chilled
547,26.0,"Mange tout, fresh, boiled",26.8,Asda Grower’s Selection mangetout,Asda Grower’s Selection,155g,16.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site: 550527 ASP1508372,Plastic,Chilled,Chilled
549,26.0,"Mange tout, fresh, boiled",26.9,The Co -operative Loved by Us trimmed mangetout,The Co -operative Loved by Us,120g,30.11.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Supplied by: M262A,Plastic,Chilled,Chilled
551,26.0,"Mange tout, fresh, boiled",26.1,Morrisons trimmed mangetout,Morrisons,155g,16.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1360/MWW T47579/0514,Plastic,Chilled,Chilled


In [190]:
Mangetout['Sub-sample Number'] = Mangetout['Sub-sample Number'].astype(str)

Match

In [191]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawmangedf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = Mange[Mange['Sub-\rsample\rnumber'].apply(lambda x: regex_matching(rawmangedf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Weight\rbefore\rcooking\r(g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
Mangetout['Pack size'] = Mangetout.apply(update_pack_size, axis=1)

In [192]:
Mangetout

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
533,26.0,"Mange tout, fresh, boiled",26.1,Tesco mangetout,Tesco,390.0,05.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HM2e,Plastic,Chilled,Chilled
535,26.0,"Mange tout, fresh, boiled",26.2,Sainsbury’s mange tout,Sainsbury’s,319.1,07.12.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,P0698D 1039405,Plastic,Chilled,Chilled
537,26.0,"Mange tout, fresh, boiled",26.3,Asda Grower’s Selection mangetout,Asda Grower’s Selection,402.0,05.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site: 550214 ASP1508372,Plastic,Chilled,Chilled
539,26.0,"Mange tout, fresh, boiled",26.4,Sainsbury’s mange tout,Sainsbury’s,417.0,05.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,P0698D 1039405 09.14,Plastic,Chilled,Chilled
541,26.0,"Mange tout, fresh, boiled",26.5,Morrisons trimmed mangetout,Morrisons,441.0,19.10.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1360/MWW,Plastic,Chilled,Chilled
543,26.0,"Mange tout, fresh, boiled",26.6,Waitrose trimmed mange tout peas,Waitrose,360.8,18.11.15,"Waitrose, Wymondham , Norfolk",22.11.15,NaN,Supplier: 343SIS 11.14,Plastic,Chilled,Chilled
545,26.0,"Mange tout, fresh, boiled",26.7,Oaklands fresh mangetout,Oaklands,402.0,05.10.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,GB005 –W4062 L4006 14:34,Plastic,Chilled,Chilled
547,26.0,"Mange tout, fresh, boiled",26.8,Asda Grower’s Selection mangetout,Asda Grower’s Selection,341.4,16.11.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site: 550527 ASP1508372,Plastic,Chilled,Chilled
549,26.0,"Mange tout, fresh, boiled",26.9,The Co -operative Loved by Us trimmed mangetout,The Co -operative Loved by Us,306.9,30.11.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Supplied by: M262A,Plastic,Chilled,Chilled
551,26.0,"Mange tout, fresh, boiled",26.1,Morrisons trimmed mangetout,Morrisons,390.0,16.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1360/MWW T47579/0514,Plastic,Chilled,Chilled


In [193]:
Mangetout.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
Mangetout.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Mangetout.csv")

##### Sample 28: Butternut squash, boiled, flesh only
Nutrient analysis survey of fresh and processed fruit and vegetables with respect to fibre
Butternut squashes were halved lengthways. The skin, ends and seeds/pulp were removed and
the flesh was cut into chunks before boiling for 15-20 minutes and drained. Each sub-sample
was randomly taken from boiled pieces from the halves of two butternut squashes.
##### Average weight of a butternut squash = 1026.4g

In [196]:
Butter = saved_dataframes[41]
Butter

,Sub-sample\rnumber,Average\rweight of a\rwhole\rsquash (g),Average\rweight of\rskin (g),Average\rweight of\rthe ends (g),Average\rweight of\rseeds (g),Edible\rportion
0,28.10,1387.2,188.0,67.4,80.3,0.76
1,28.20,1014.1,153.1,13.7,51.8,0.78
2,28.30,1101.1,145.8,53.7,44.2,0.78
3,28.40,1017.8,135.3,24.7,40.5,0.80
4,28.50,588.9,86.2,16.6,37.2,0.76
5,28.60,1439.9,213.7,18.5,48.5,0.81
6,28.70,967.9,116.3,65.1,84.8,0.72
7,28.80,N/A,N/A,N/A,N/A,N/A
8,28.90,660.2,94.8,34.3,39.4,0.74
9,28.10,1319.4,210.9,42.4,72.4,0.75


In [197]:
Butter['Sub-sample\rnumber'] = Butter['Sub-sample\rnumber'].astype(str)

In [200]:
Butternut  = filtered_products[filtered_products['Composite Sample Name'].str.contains('Butternut ')][:11]
Butternut 

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
577,28.0,"Butternut squash, boiled flesh only",28.1,Tesco roasting butternut squash,Tesco,Purchase Date:,05.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:AGS01,Loose,Ambient,21ºC controlled temperature room
579,28.0,"Butternut squash, boiled flesh only",28.2,Tesco butternut squash,Tesco,Purchase Date:,27.01.16,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
581,28.0,"Butternut squash, boiled flesh only",28.3,Asda Grower’s Selection butternut squash,Asda Grower’s Selection,Purchase Date:,05.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:909750 PO:442534 258 PLU643 ASP1510543,Loose,Ambient,21ºC controlled temperature room
583,28.0,"Butternut squash, boiled flesh only",28.4,Sainsbury’s butternut squash,Sainsbury’s,Purchase Date:,13.01.16,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,B0090 B 009 6156 14 31 273,Loose,Ambient,21ºC controlled temperature room
585,28.0,"Butternut squash, boiled flesh only",28.5,Morrisons butternut squash,Morrisons,Purchase Date:,19.10.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1360/MWW,Loose,Ambient,21ºC controlled temperature room
587,28.0,"Butternut squash, boiled flesh only",28.6,Essential Waitrose butternut squash,Essential Waitrose,Purchase Date:,16.01.16,"Waitrose, Wymondham , Norfolk",31.01.16,NaN,1687 Supplier: 111AGT 09:14,Loose,Ambient,21°C controlled temperature room
589,28.0,"Butternut squash, boiled flesh only",28.7,Lidl butternut squash,Lidl,Purchase Date:,05.10.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
591,28.0,"Butternut squash, boiled flesh only",28.8,Sainsbury’s ready prepared butternut squash,Sainsbury’s,320g,26.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Use By Date:,NaN,B0090 B UP1 5 07:09 7355010 01.15,Plastic,Chilled,Chilled
593,28.0,"Butternut squash, boiled flesh only",28.9,The Co -operative butternut squash,The Co -operative,Purchase Date:,05.01.16,"The Co -operative, Wymondham , Norfolk",Best before Date:,NaN,M244 P8635/1/2,Loose,Ambient,21ºC controlled temperature room
595,28.0,"Butternut squash, boiled flesh only",28.1,Market butternut squash,Pack size:,Purchase Date:,08.10.15,NaN,Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room


In [201]:
Butternut['Sub-sample Number'] = Butternut['Sub-sample Number'].astype(str)

Match

In [202]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawbuttdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = Butter[Butter['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawbuttdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole\rsquash (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
Butternut['Pack size'] = Butternut.apply(update_pack_size, axis=1)

In [203]:
Butternut

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
577,28.0,"Butternut squash, boiled flesh only",28.1,Tesco roasting butternut squash,Tesco,1387.2,05.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:AGS01,Loose,Ambient,21ºC controlled temperature room
579,28.0,"Butternut squash, boiled flesh only",28.2,Tesco butternut squash,Tesco,1014.1,27.01.16,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
581,28.0,"Butternut squash, boiled flesh only",28.3,Asda Grower’s Selection butternut squash,Asda Grower’s Selection,1101.1,05.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:909750 PO:442534 258 PLU643 ASP1510543,Loose,Ambient,21ºC controlled temperature room
583,28.0,"Butternut squash, boiled flesh only",28.4,Sainsbury’s butternut squash,Sainsbury’s,1017.8,13.01.16,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,B0090 B 009 6156 14 31 273,Loose,Ambient,21ºC controlled temperature room
585,28.0,"Butternut squash, boiled flesh only",28.5,Morrisons butternut squash,Morrisons,588.9,19.10.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1360/MWW,Loose,Ambient,21ºC controlled temperature room
587,28.0,"Butternut squash, boiled flesh only",28.6,Essential Waitrose butternut squash,Essential Waitrose,1439.9,16.01.16,"Waitrose, Wymondham , Norfolk",31.01.16,NaN,1687 Supplier: 111AGT 09:14,Loose,Ambient,21°C controlled temperature room
589,28.0,"Butternut squash, boiled flesh only",28.7,Lidl butternut squash,Lidl,967.9,05.10.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
591,28.0,"Butternut squash, boiled flesh only",28.8,Sainsbury’s ready prepared butternut squash,Sainsbury’s,N/A,26.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Use By Date:,NaN,B0090 B UP1 5 07:09 7355010 01.15,Plastic,Chilled,Chilled
593,28.0,"Butternut squash, boiled flesh only",28.9,The Co -operative butternut squash,The Co -operative,660.2,05.01.16,"The Co -operative, Wymondham , Norfolk",Best before Date:,NaN,M244 P8635/1/2,Loose,Ambient,21ºC controlled temperature room
595,28.0,"Butternut squash, boiled flesh only",28.1,Market butternut squash,Pack size:,1387.2,08.10.15,NaN,Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room


In [204]:
Butternut.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
Butternut.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\ButternutSquash.csv")

##### Sample 30: Avocado pear, raw, flesh only
The skin and stone were removed from avocados before sampling. Each sub-sample was
taken from the halves of 2-3 avocados.
##### Average weight of an avocado = 214.6g

In [213]:
Avo = saved_dataframes[45]#[:9]
Avo

,Sub-sample\rnumber,Average\rweight of a\rwhole\ravocado (g),Average\rweight of\rskin (g),Average\rweight of\rstone (g),Edible\rportion


In [214]:
Avocado = saved_dataframes[46]

In [215]:
Avocado.columns = Avo.columns

In [223]:
Avocado = saved_dataframes[46][:8]
Avocado

,Sub-sample\rnumber,Average\rweight of a\rwhole\ravocado (g),Average\rweight of\rskin (g),Average\rweight of\rstone (g),Edible\rportion
0,30.2,209.0,26.2,37.3,0.70
1,30.3,172.9,25.5,16.5,0.76
2,30.4,283.5,31.7,56.4,0.69
3,30.5,286.4,37.8,36.3,0.74
4,30.6,185.5,29.9,29.0,0.68
5,30.7,264.3,47.8,39.3,0.67
6,30.8,169.1,23.8,30.5,0.68
7,30.9,179.6,26.9,24.7,0.71


In [218]:
Avocado['Sub-sample\rnumber'] = Avocado['Sub-sample\rnumber'].astype(str)

In [226]:
Avocadopear  = filtered_products[filtered_products['Composite Sample Name'].str.contains('Avocado')][:9]
Avocadopear

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
621,30.0,"Avocado pear, raw, flesh only",30.1,Tesco ripen at home avocados,Tesco,4 fruit,05.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:WES02,Plastic,Ambient,21ºC controlled temperature room
623,30.0,"Avocado pear, raw, flesh only",30.2,Tesco large avocado,Tesco,Purchase Date:,07.12.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:WES02,Sample Condition As Purchased:,Chilled,Chilled
625,30.0,"Avocado pear, raw, flesh only",30.3,Asda Grower’s Selection ripe and ready avocado,Asda Grower’s Selection,2 fruit,NaN,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:253446 X2 ASP1418697,Plastic,Chilled,Chilled
627,30.0,"Avocado pear, raw, flesh only",30.4,Sainsbury’s extra large avocado,Sainsbury’s,Purchase Date:,05.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,G1036 7678882 07.15,Loose,Ambient,Chilled
629,30.0,"Avocado pear, raw, flesh only",30.5,Morrisons avocados,Morrisons,600g,07.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1230/UUK 2725 T59013/0915 LA53705700,Plastic,Ambient,21ºC controlled temperature room
631,30.0,"Avocado pear, raw, flesh only",30.6,M&S Hass avocado,Marks & Spencer,Purchase Date:,05.10.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,1826,Plastic; paper tray,Ambient,Chilled
633,30.0,"Avocado pear, raw, flesh only",30.7,Lidl avocado,Lidl,Purchase Date:,NaN,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
635,30.0,"Avocado pear, raw, flesh only",30.8,Morrisons ready to eat avocados,Morrisons,Count 2,07.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1230 T47609/0514,Plastic,Chilled,Chilled
637,30.0,"Avocado pear, raw, flesh only",30.9,The Co -operative Loved by Us 2 Avocados,The Co -operative Loved by Us,Purchase Date:,07.12.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Supplied by: M199 P14741/1/1,Plastic,Chilled,Chilled


In [227]:
Avocadopear['Sub-sample Number'] = Avocadopear['Sub-sample Number'].astype(str)

Match

In [228]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawavacadf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = Avocado[Avocado['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawavacadf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole\ravocado (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
Avocadopear['Pack size'] = Avocadopear.apply(update_pack_size, axis=1)

In [229]:
Avocadopear

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
621,30.0,"Avocado pear, raw, flesh only",30.1,Tesco ripen at home avocados,Tesco,4 fruit,05.10.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:WES02,Plastic,Ambient,21ºC controlled temperature room
623,30.0,"Avocado pear, raw, flesh only",30.2,Tesco large avocado,Tesco,209.0,07.12.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:WES02,Sample Condition As Purchased:,Chilled,Chilled
625,30.0,"Avocado pear, raw, flesh only",30.3,Asda Grower’s Selection ripe and ready avocado,Asda Grower’s Selection,172.9,NaN,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:253446 X2 ASP1418697,Plastic,Chilled,Chilled
627,30.0,"Avocado pear, raw, flesh only",30.4,Sainsbury’s extra large avocado,Sainsbury’s,283.5,05.10.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,G1036 7678882 07.15,Loose,Ambient,Chilled
629,30.0,"Avocado pear, raw, flesh only",30.5,Morrisons avocados,Morrisons,286.4,07.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1230/UUK 2725 T59013/0915 LA53705700,Plastic,Ambient,21ºC controlled temperature room
631,30.0,"Avocado pear, raw, flesh only",30.6,M&S Hass avocado,Marks & Spencer,185.5,05.10.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,1826,Plastic; paper tray,Ambient,Chilled
633,30.0,"Avocado pear, raw, flesh only",30.7,Lidl avocado,Lidl,264.3,NaN,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
635,30.0,"Avocado pear, raw, flesh only",30.8,Morrisons ready to eat avocados,Morrisons,169.1,07.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,1230 T47609/0514,Plastic,Chilled,Chilled
637,30.0,"Avocado pear, raw, flesh only",30.9,The Co -operative Loved by Us 2 Avocados,The Co -operative Loved by Us,179.6,07.12.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Supplied by: M199 P14741/1/1,Plastic,Chilled,Chilled


In [230]:
Avocadopear.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
Avocadopear.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Avocadopear.csv")

##### Sample 31: Sweet potato, boiled, flesh only
Sweet potatoes were peeled, cut into chunks and rinsed in Analar water before boiling for 5-20
minutes and draining. Each sub-sample was randomly taken from boiled pieces from the halves
of three sweet potatoes.
##### Average weight of a sweet potato = 249.3g

In [231]:
sweetpot = saved_dataframes[47]
sweetpot

,Sub-sample\rnumber,Average\rweight of a\rwhole sweet\rpotato (g),Average\rweight of\rskin (g),Edible\rportion
0,31.10,142.0,21.7,0.85
1,31.20,187.0,23.9,0.87
2,31.30,261.7,28.7,0.89
3,31.40,186.0,26.3,0.86
4,31.50,129.3,19.3,0.85
5,31.60,188.7,25.0,0.87
6,31.70,403.7,62.3,0.85
7,31.80,316.7,35.7,0.89
8,31.90,233.7,29.7,0.87
9,31.10,426.0,42.2,0.90


In [232]:
sweetpot['Sub-sample\rnumber'] = sweetpot['Sub-sample\rnumber'].astype(str)

In [235]:
potato  = filtered_products[filtered_products['Composite Sample Name'].str.contains('Sweet potato')][:11]
potato

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
643,31.0,"Sweet potato, boiled, flesh only",31.1,Tesco sweet potatoes,Tesco,1kg,07.12.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HY01(D) 4815IH,Plastic,Ambient,21ºC controlled temperature room
645,31.0,"Sweet potato, boiled, flesh only",31.2,Tesco sweet potatoes,Tesco,1kg,05.01.16,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HY01(D) 5215CL,Plastic,Ambient,21ºC controlled temperature room
647,31.0,"Sweet potato, boiled, flesh only",31.3,Asda Grower’s Selection sweet potatoes,Asda Grower’s Selection,1kg,07.12.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,16:47 Batch: 794077 Site:900326 ASP1501783,Plastic,Ambient,21ºC controlled temperature room
649,31.0,"Sweet potato, boiled, flesh only",31.4,Sainsbury’s sweet potato,Sainsbury’s,1.25kg,07.12.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,B0090B 1191417 11.14,Plastic netting,Ambient,21ºC controlled temperature room
651,31.0,"Sweet potato, boiled, flesh only",31.5,Morrisons sweet potato,Morrisons,Purchase Date:,07.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
653,31.0,"Sweet potato, boiled, flesh only",31.6,M&S sweet potatoes,Marks & Spencer,1kg,07.12.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC2045,Plastic,Ambient,21ºC controlled tempearature room
655,31.0,"Sweet potato, boiled, flesh only",31.7,Lidl sweet potatoes,Lidl,Purchase Date:,07.12.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
657,31.0,"Sweet potato, boiled, flesh only",31.8,Morrisons sweet potatoes,Morrisons,Min. 5,07.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,226123 TH1JOK 3 40 09:30 ED BG53700584 8600,Plastic,Ambient,21ºC controlled temperature room
659,31.0,"Sweet potato, boiled, flesh only",31.9,The Co -operative sweet potatoes,The Co -operative,Purchase Date:,07.12.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
661,31.0,"Sweet potato, boiled, flesh only",31.1,Greengrocer sweet potatoes,Pack size:,Purchase Date:,09.12.15,"Independent greengrocer, Gloucester Road, Norw...",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room


In [236]:
potato['Sub-sample Number'] = potato['Sub-sample Number'].astype(str)

Match

In [238]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawsweetpdf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = sweetpot[sweetpot['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawsweetpdf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole sweet\rpotato (g)']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
potato['Pack size'] = potato.apply(update_pack_size, axis=1)

In [239]:
potato

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
643,31.0,"Sweet potato, boiled, flesh only",31.1,Tesco sweet potatoes,Tesco,142.0,07.12.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HY01(D) 4815IH,Plastic,Ambient,21ºC controlled temperature room
645,31.0,"Sweet potato, boiled, flesh only",31.2,Tesco sweet potatoes,Tesco,187.0,05.01.16,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC:HY01(D) 5215CL,Plastic,Ambient,21ºC controlled temperature room
647,31.0,"Sweet potato, boiled, flesh only",31.3,Asda Grower’s Selection sweet potatoes,Asda Grower’s Selection,261.7,07.12.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,16:47 Batch: 794077 Site:900326 ASP1501783,Plastic,Ambient,21ºC controlled temperature room
649,31.0,"Sweet potato, boiled, flesh only",31.4,Sainsbury’s sweet potato,Sainsbury’s,186.0,07.12.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,B0090B 1191417 11.14,Plastic netting,Ambient,21ºC controlled temperature room
651,31.0,"Sweet potato, boiled, flesh only",31.5,Morrisons sweet potato,Morrisons,129.3,07.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
653,31.0,"Sweet potato, boiled, flesh only",31.6,M&S sweet potatoes,Marks & Spencer,188.7,07.12.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC2045,Plastic,Ambient,21ºC controlled tempearature room
655,31.0,"Sweet potato, boiled, flesh only",31.7,Lidl sweet potatoes,Lidl,403.7,07.12.15,"Lidl, Drayton Road, Norwich",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
657,31.0,"Sweet potato, boiled, flesh only",31.8,Morrisons sweet potatoes,Morrisons,316.7,07.12.15,"Morrisons, Albion Way, Norwich",Best before Date:,NaN,226123 TH1JOK 3 40 09:30 ED BG53700584 8600,Plastic,Ambient,21ºC controlled temperature room
659,31.0,"Sweet potato, boiled, flesh only",31.9,The Co -operative sweet potatoes,The Co -operative,233.7,07.12.15,"The Co -operative, Long Stratton , Norfolk",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room
661,31.0,"Sweet potato, boiled, flesh only",31.1,Greengrocer sweet potatoes,Pack size:,142.0,09.12.15,"Independent greengrocer, Gloucester Road, Norw...",Best before Date:,NaN,Packaging:,Loose,Ambient,21ºC controlled temperature room


In [240]:
potato.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
potato.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Sweetpotato.csv")

##### Sample 33: Broad beans, fresh, boiled
Broad beans were removed from their pods before being boiled for 3-10 minutes and drained.
Each sub-sample was randomly taken from around 100 boiled beans.
##### Average weight of a broad bean (including pod and beans) = 42.3g

In [248]:
bb = saved_dataframes[51]
bb

,Sub-sample\rnumber,Average\rweight of a\rwhole broad,Average\rweight of\rpod (g),Average\rweight of\rraw bean,Edible\rportion


In [249]:
bb1 = saved_dataframes[52]

In [250]:
bb1.columns = bb.columns

In [251]:
bb1

,Sub-sample\rnumber,Average\rweight of a\rwhole broad,Average\rweight of\rpod (g),Average\rweight of\rraw bean,Edible\rportion
0,33.10,39.2,29.4,2.3,0.25
1,33.20,47.6,33.9,3.4,0.29
2,33.30,74.7,37.6,2.6,0.50
3,33.40,32.0,22.5,2.4,0.30
4,33.50,32.0,20.0,2.6,0.37
5,33.60,29.3,19.8,3.1,0.32
6,33.70,45.1,31.9,3.8,0.29
7,33.80,42.0,32.2,2.8,0.23
8,33.90,42.3,30.3,2.6,0.28
9,33.10,45.0,29.6,3.5,0.34


In [253]:
bb1['Sub-sample\rnumber'] = bb1['Sub-sample\rnumber'].astype(str)

In [252]:
Broadb  = filtered_products[filtered_products['Composite Sample Name'].str.contains('Broad beans')]#[:11]
Broadb

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
687,33.0,"Broad beans, fresh, boiled",33.1,Tesco British broad beans,Tesco,500g,11.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,S4 6 12:52 SC 212977 (BOB01),Plastic,Chilled,Chilled
689,33.0,"Broad beans, fresh, boiled",33.2,Tesco British broad beans,Tesco,500g,17.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,S4 2 11:54 SC 212977 (BOB01),Plastic,Chilled,Chilled
691,33.0,"Broad beans, fresh, boiled",33.3,Tesco broad beans,Tesco,500g,22.09.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC 212977 (BOB01),Plastic,Chilled,Chilled
693,33.0,"Broad beans, fresh, boiled",33.4,Asda Grower’s Selection broad beans,Asda Grower’s Selection,500g,11.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:550307 ASP1507907,Plastic,Chilled,Chilled
695,33.0,"Broad beans, fresh, boiled",33.5,Sainsbury’s broad beans,Sainsbury’s,500g,11.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,K0022 1383478 04.14,Plastic,Chilled,Chilled
697,33.0,"Broad beans, fresh, boiled",33.6,Morrisons broad beans,Morrisons,500g,11.08.15,"Morrisons, Albion way, Norwich",Best before Date:,NaN,2624 T47598/0514,Plastic,Chilled,Chilled
699,33.0,"Broad beans, fresh, boiled",33.7,Essential Waitrose broad beans,Essential Waitrose,500g,11.08.15,"Waitrose, Eaton , Norwich",13.08.15,NaN,284AAS 04.15,Plastic,Chilled,Chilled
701,33.0,"Broad beans, fresh, boiled",33.8,Asda Grower’s Selection British broad beans,Asda Grower’s Selection,500g,19.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:550522 ASP1419787,Plastic,NaN,Chilled
703,33.0,"Broad beans, fresh, boiled",33.9,M&S British broad beans,Marks & Spencer,450g,19.10.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC4146,Plastic,Chilled,Chilled
705,33.0,"Broad beans, fresh, boiled",33.1,Tesco broad beans,Tesco,500g,16.10.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC:WD03,Plastic,Chilled,Chilled


In [254]:
Broadb['Sub-sample Number'] = Broadb['Sub-sample Number'].astype(str)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\3063366098.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Broadb['Sub-sample Number'] = Broadb['Sub-sample Number'].astype(str)


Match

In [255]:
# Define your regular expression matching function
def regex_matching(source, target):
    pattern = re.compile(f'.*{source}.*', re.IGNORECASE)
    return target if re.match(pattern, target) else ''

# Create a function to update 'Pack size' based on the regex match
def update_pack_size(row):
    rawbroaddf = row['Sub-sample Number']
    
    # Find the best match in 'Kiwi' DataFrame using regular expression
    best_match = bb1[bb1['Sub-sample\rnumber'].apply(lambda x: regex_matching(rawbroaddf, x)) != '']
    
    if not best_match.empty:
        # Get the first match and update 'Pack size'
        matched_weight = best_match.iloc[0]['Average\rweight of a\rwhole broad']
        return matched_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'rawKiw'
Broadb['Pack size'] = Broadb.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\85082139.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Broadb['Pack size'] = Broadb.apply(update_pack_size, axis=1)


In [256]:
Broadb

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
687,33.0,"Broad beans, fresh, boiled",33.1,Tesco British broad beans,Tesco,39.2,11.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,S4 6 12:52 SC 212977 (BOB01),Plastic,Chilled,Chilled
689,33.0,"Broad beans, fresh, boiled",33.2,Tesco British broad beans,Tesco,47.6,17.08.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,S4 2 11:54 SC 212977 (BOB01),Plastic,Chilled,Chilled
691,33.0,"Broad beans, fresh, boiled",33.3,Tesco broad beans,Tesco,74.7,22.09.15,"Tesco, Harford Bridges, Norwich",Best before Date:,NaN,SC 212977 (BOB01),Plastic,Chilled,Chilled
693,33.0,"Broad beans, fresh, boiled",33.4,Asda Grower’s Selection broad beans,Asda Grower’s Selection,32.0,11.08.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:550307 ASP1507907,Plastic,Chilled,Chilled
695,33.0,"Broad beans, fresh, boiled",33.5,Sainsbury’s broad beans,Sainsbury’s,32.0,11.08.15,"Sainsbury’s, Longwater Retail Park, Norwich",Best before Date:,NaN,K0022 1383478 04.14,Plastic,Chilled,Chilled
697,33.0,"Broad beans, fresh, boiled",33.6,Morrisons broad beans,Morrisons,29.3,11.08.15,"Morrisons, Albion way, Norwich",Best before Date:,NaN,2624 T47598/0514,Plastic,Chilled,Chilled
699,33.0,"Broad beans, fresh, boiled",33.7,Essential Waitrose broad beans,Essential Waitrose,45.1,11.08.15,"Waitrose, Eaton , Norwich",13.08.15,NaN,284AAS 04.15,Plastic,Chilled,Chilled
701,33.0,"Broad beans, fresh, boiled",33.8,Asda Grower’s Selection British broad beans,Asda Grower’s Selection,42.0,19.10.15,"Asda, Boundary Road, Norwich",Best before Date:,NaN,Site:550522 ASP1419787,Plastic,NaN,Chilled
703,33.0,"Broad beans, fresh, boiled",33.9,M&S British broad beans,Marks & Spencer,42.3,19.10.15,"Marks & Spencer, Sweet Briar Retail Park, Norw...",Best before Date:,NaN,SC4146,Plastic,Chilled,Chilled
705,33.0,"Broad beans, fresh, boiled",33.1,Tesco broad beans,Tesco,39.2,16.10.15,"Tesco Extra, Blue Boar Road, Norwich",Best before Date:,NaN,SC:WD03,Plastic,Chilled,Chilled


In [257]:
Broadb.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
Broadb.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\2015\Broadbeans.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_25840\2929243383.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Broadb.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


##### Sample 34: Curly kale, fresh, boiled
The larger stem and tough ribs were removed from the curly kale and rinsed in Analar water
before being boiled for 3-9 minutes and drained thoroughly. Each sub-sample was randomly
taken from about nine boiled leaves
##### Average weight of a curly kale leaf = 42.3g

In [261]:
Curly = saved_dataframes[54]#[:9]
Curly

,Sub-sample\rnumber,Average\rweight of a\rwhole leaf\r(g),Average\rweight of\rtough\rmiddle\rstalk(g),Edible\rportion
0,34.10,46.2,18.6,0.6
1,34.11,38.4,11.4,0.7


In [262]:
kale = saved_dataframes[55]
kale

,Sub-\rsample\rnumber,Boiling\rtime\r(minutes),Weight\rbefore\rcooking\r(g),Weight\rafter\rcooking\r(g),Weight\rchange\r(%)
0,34.10,7,246.7,298.7,21.1
1,34.20,7,245.8,312.8,27.3
2,34.30,9,267.6,343.7,28.4
3,34.40,7,295.5,377.7,27.8
4,34.50,9,243.6,301.0,23.6
5,34.60,7,177.3,228.9,29.1
6,34.70,3,186.5,197.0,5.6
7,34.80,8,247.8,273.8,10.5
8,34.90,4,232.1,298.2,28.5
9,34.10,3,231.0,355.0,53.7


In [111]:
per = filtered_products[698:707]
per

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
698,42.0,"Pears, raw, flesh and skin",42.1,"Tesco, loose Conference pears",Pack size:,Purchase Date: 18.11.2011,18.11.2011,"Tesco, Blue B oar Road, Norwich",Best Before Date:,Batch Number:,Packaging: Plastic,Plastic,Ambient,Ambient
699,42.0,"Pears, raw, flesh and skin",42.2,"Tesco, pears",NaN,Count: 5 fruit,18.11.2011,"Tesco, Blue Boar Road , Norwich",23-Nov,25-Nov,L15 kcb 3454 EW 100,Plastic,NaN,Ambient
700,42.0,"Pears, raw, flesh and skin",42.3,"Asda, loose Rocha pears",Pack size:,Purchase Date: 18.11.201 1,18.11.201 1,"Asda , Norwich",Best Before Date:,Batch Number: 3024,3024,Plastic,Ambient,Ambient
701,42.0,"Pears, raw, flesh and skin",42.4,"Asda, loose Conference pears",Pack size:,Purchase Date: 18.11.2011,18.11.2011,"Asda , Norwich",Best Before Date:,Batch Number:,Packaging: Plastic,Plastic,Ambient,NaN
702,42.0,"Pears, raw, flesh and skin",42.5,"Sainsbury’s, Conference pears",Sainsbury’s,7 pears,18.11.2011,"Sainsbury’s, Longwater , Norwich",Best Before Date: 27 Nov,27-Nov,O0218C/19 117975 09.11,Plastic,Ambient,Ambient
703,42.0,"Pears, raw, flesh and skin",42.6,"Morrisons, Rocha pears",Morrisons,Purchase Date: 23.11.2011,23.11.2011,"Morrisons , Norwich",24-Nov,26-Nov,26130B TH1 SOR2 322 1123:36 PJ,NaN,Ambient,Ambient
704,42.0,"Pears, raw, flesh and skin",42.7,"Marks & Spencer, perfectly ripe Concorde pears",Marks & Spencer,Purchase Date: 23.11.2011,23.11.2011,"Marks & Spencer , Norwich",25-Nov,Batch Number: 4508 B,4508 B,Plastic,Ambient,Ambient
705,42.0,"Pears, raw, flesh and skin",42.8,"Aldi, Conference pears",Aldi,8 fruits,18.11.2011,"Aldi, Norwich",Best Before Date:,Batch Number: Supplier code: 1117 Defra code: ...,Supplier code: 1117 Defra code: E58530 4701 EW86,Plastic,Ambient,Ambient
706,42.0,"Pears, raw, flesh and skin",42.9,"Sainsbury’s, Basics, pears",Sainsbury’s Basics,500g,18.11.2011,"Sainsbury’s, Longwater , Norwich",Best Before Date: 27 Nov,27-Nov,O0218V(P) 1125696 08.09,Plastic,Ambient,Ambient


Match

In [112]:
# Define your fuzzy matching function
def fuzzy_matching_score(source, target):
    return fuzz.token_sort_ratio(source, target) 

# Create a function to update 'Pack size' based on the match
def update_pack_size(row):
    cartdf = row['Sub-sample Number']
    
    # Find the best match in 'a' dataframe
    matches = pear[pear['Sub-sample\rnumber'].apply(lambda x: fuzzy_matching_score(cartdf, x)) >= 80]
    
    if not matches.empty:
        # Get the matched weights and select the highest one
        matched_weights = matches['Average weight\rwhole pear (g)']
        best_match_weight = matched_weights.max()
        return best_match_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'filtered_products'
per['Pack size'] = per.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\487621355.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per['Pack size'] = per.apply(update_pack_size, axis=1)


In [113]:
per

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
698,42.0,"Pears, raw, flesh and skin",42.1,"Tesco, loose Conference pears",Pack size:,201.8,18.11.2011,"Tesco, Blue B oar Road, Norwich",Best Before Date:,Batch Number:,Packaging: Plastic,Plastic,Ambient,Ambient
699,42.0,"Pears, raw, flesh and skin",42.2,"Tesco, pears",NaN,177.8,18.11.2011,"Tesco, Blue Boar Road , Norwich",23-Nov,25-Nov,L15 kcb 3454 EW 100,Plastic,NaN,Ambient
700,42.0,"Pears, raw, flesh and skin",42.3,"Asda, loose Rocha pears",Pack size:,139.0,18.11.201 1,"Asda , Norwich",Best Before Date:,Batch Number: 3024,3024,Plastic,Ambient,Ambient
701,42.0,"Pears, raw, flesh and skin",42.4,"Asda, loose Conference pears",Pack size:,278.7,18.11.2011,"Asda , Norwich",Best Before Date:,Batch Number:,Packaging: Plastic,Plastic,Ambient,NaN
702,42.0,"Pears, raw, flesh and skin",42.5,"Sainsbury’s, Conference pears",Sainsbury’s,146.7,18.11.2011,"Sainsbury’s, Longwater , Norwich",Best Before Date: 27 Nov,27-Nov,O0218C/19 117975 09.11,Plastic,Ambient,Ambient
703,42.0,"Pears, raw, flesh and skin",42.6,"Morrisons, Rocha pears",Morrisons,120.5,23.11.2011,"Morrisons , Norwich",24-Nov,26-Nov,26130B TH1 SOR2 322 1123:36 PJ,NaN,Ambient,Ambient
704,42.0,"Pears, raw, flesh and skin",42.7,"Marks & Spencer, perfectly ripe Concorde pears",Marks & Spencer,246.4,23.11.2011,"Marks & Spencer , Norwich",25-Nov,Batch Number: 4508 B,4508 B,Plastic,Ambient,Ambient
705,42.0,"Pears, raw, flesh and skin",42.8,"Aldi, Conference pears",Aldi,99.0,18.11.2011,"Aldi, Norwich",Best Before Date:,Batch Number: Supplier code: 1117 Defra code: ...,Supplier code: 1117 Defra code: E58530 4701 EW86,Plastic,Ambient,Ambient
706,42.0,"Pears, raw, flesh and skin",42.9,"Sainsbury’s, Basics, pears",Sainsbury’s Basics,204.9,18.11.2011,"Sainsbury’s, Longwater , Norwich",Best Before Date: 27 Nov,27-Nov,O0218V(P) 1125696 08.09,Plastic,Ambient,Ambient


In [114]:
per.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
per.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\Pear.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\1078661602.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  per.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


                                                      Strawberry

In [115]:
strawberry = saved_dataframes[70][:9]
strawberry

,Sub-sample\rnumber,Average\rweight whole\rstrawberry (g),Average\rweight stalk,Edible\rportion
0,45.1,13.1,0.5,0.96
1,45.2,16.7,1.9,0.89
2,45.3,16.7,1.2,0.93
3,45.4,11.6,0.4,0.97
4,45.5,20.1,1.2,0.94
5,45.6,17.7,1.7,0.90
6,45.7,15.9,0.8,0.95
7,45.8,15.2,0.9,0.94
8,45.9,17.4,1.4,0.92


In [116]:
straw = filtered_products[758:767]
straw

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
758,NaN,Sub-sample Number:,Sub-sample Name:,Brand:,Pack size:,Purchase Date:,Place of Purchase:,Displa y Until Date:,NaN,Batch Number:,Packaging:,Sample Condition As Purchased:,Post Purchase Storage:,45
759,45.0,"Strawberries, raw",45.2,NaN,Tesco Finest,300g,23.08.2011,"Tesco, Harford Bridges , Norwich",25-Aug,Batch Number: KG20,KG20,Plastic,Ambient,Refrigerated
760,45.0,"Strawberries, raw",45.3,"Asda, strawberries",Asda,400g,23.08.2011,"Asda , Norwich",23-Aug,24-Aug,900790 141820 ASL1001831,Plastic,Ambient,Refrigerated
761,45.0,"Strawberries, raw",45.4,"Asda, strawberries",Asda,300g,23.08.2011,"Asda , Norwich",25-Aug,26-Aug,571045 ASL1001228,Plastic,Ambient,Refrigerated
762,45.0,"Strawberries, raw",45.5,"Sainsbury’s, British strawberries",Asda,400g,23.08.2011,"Sainsbury’s, Longwater , Norwich",27-Aug,27-Aug,A0800M,NaN,Ambient,Refrigerated
763,45.0,"Strawberries, raw",45.6,"Morrisons, strawberries",Morrisons,400g,NaN,"Morrisons , Norwich",25-Aug,26-Aug,1301/13 T33160/0111,Plastic,Ambient,Refrigerated
764,45.0,"Strawberries, raw",45.7,"Marks & Spencer, juicy strawberries",Marks & Spencer,400g,24.08.2011,"Marks & Spencer , Norwich",24-Aug,Batch Number: 4016G,4016G,Plastic,Ambient,Refrigerated
765,45.0,"Strawberries, raw",45.8,"Oaklands, British strawberries",Oaklands,400g,22.08.2011,"Lidl, Norwich",Best Before Date:,Batch Number: 3304 W2416,3304 W2416,Plastic,Ambient,Refrigerated
766,45.0,"Strawberries, raw",45.9,"Tesco, Market Value, str awberries",Tesco Market Value,400g,23.08.2011,"Tesco, Harford Bridges, Norwich",25-Aug,Batch Number: SAD1,SAD1,Plastic,NaN,Refrigerated


Match

In [117]:
# Define your fuzzy matching function
def fuzzy_matching_score(source, target):
    return fuzz.token_sort_ratio(source, target) 

# Create a function to update 'Pack size' based on the match
def update_pack_size(row):
    cartdf = row['Sub-sample Number']
    
    # Find the best match in 'a' dataframe
    matches = strawberry[strawberry['Sub-sample\rnumber'].apply(lambda x: fuzzy_matching_score(cartdf, x)) >= 80]
    
    if not matches.empty:
        # Get the matched weights and select the highest one
        matched_weights = matches['Average\rweight whole\rstrawberry (g)']
        best_match_weight = matched_weights.max()
        return best_match_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'filtered_products'
straw['Pack size'] = straw.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\1675714717.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  straw['Pack size'] = straw.apply(update_pack_size, axis=1)


In [118]:
straw

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
758,NaN,Sub-sample Number:,Sub-sample Name:,Brand:,Pack size:,Purchase Date:,Place of Purchase:,Displa y Until Date:,NaN,Batch Number:,Packaging:,Sample Condition As Purchased:,Post Purchase Storage:,45
759,45.0,"Strawberries, raw",45.2,NaN,Tesco Finest,16.7,23.08.2011,"Tesco, Harford Bridges , Norwich",25-Aug,Batch Number: KG20,KG20,Plastic,Ambient,Refrigerated
760,45.0,"Strawberries, raw",45.3,"Asda, strawberries",Asda,16.7,23.08.2011,"Asda , Norwich",23-Aug,24-Aug,900790 141820 ASL1001831,Plastic,Ambient,Refrigerated
761,45.0,"Strawberries, raw",45.4,"Asda, strawberries",Asda,11.6,23.08.2011,"Asda , Norwich",25-Aug,26-Aug,571045 ASL1001228,Plastic,Ambient,Refrigerated
762,45.0,"Strawberries, raw",45.5,"Sainsbury’s, British strawberries",Asda,20.1,23.08.2011,"Sainsbury’s, Longwater , Norwich",27-Aug,27-Aug,A0800M,NaN,Ambient,Refrigerated
763,45.0,"Strawberries, raw",45.6,"Morrisons, strawberries",Morrisons,17.7,NaN,"Morrisons , Norwich",25-Aug,26-Aug,1301/13 T33160/0111,Plastic,Ambient,Refrigerated
764,45.0,"Strawberries, raw",45.7,"Marks & Spencer, juicy strawberries",Marks & Spencer,15.9,24.08.2011,"Marks & Spencer , Norwich",24-Aug,Batch Number: 4016G,4016G,Plastic,Ambient,Refrigerated
765,45.0,"Strawberries, raw",45.8,"Oaklands, British strawberries",Oaklands,15.2,22.08.2011,"Lidl, Norwich",Best Before Date:,Batch Number: 3304 W2416,3304 W2416,Plastic,Ambient,Refrigerated
766,45.0,"Strawberries, raw",45.9,"Tesco, Market Value, str awberries",Tesco Market Value,17.4,23.08.2011,"Tesco, Harford Bridges, Norwich",25-Aug,Batch Number: SAD1,SAD1,Plastic,NaN,Refrigerated


In [119]:
straw.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
straw.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\Strawberry.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\2743157204.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  straw.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


                                                        Oranges

In [120]:
oranges = saved_dataframes[72][:9]
oranges

,Sub-sample\rnumber,Average weight\rwhole fruit (g),Average\rweight peel (g),Average\rweight pips (g),Average weight\r‘pithy core’1,Edible\rportion
0,48.1,271.3,81.2,0.0,NaN,0.70
1,48.2,240.1,61.5,0.0,NaN,0.74
2,48.3,224.9,64.2,0.0,NaN,0.71
3,48.4,217.6,51.2,0.0,NaN,0.76
4,48.5,308.3,78.8,0.0,NaN,0.74
5,48.6,273.7,75.4,0.0,NaN,0.72
6,48.7,182.3,45.9,1.1,NaN,0.74
7,48.8,175.7,43.9,1.2,NaN,0.74
8,48.9,151.6,41.3,0.0,NaN,0.73


In [121]:
rang = filtered_products[817:824]
rang

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
817,48.0,"Oranges, flesh only",48.1,"Tesco, large oranges",Tesco,5 fruit,21.11.201 1,"Tesco, Blue Boar Road, Norwich",25-Nov,27-Nov,WO:43258 MZ16 TC:33101889 16:52 XO2,Netting,Ambient,Refrigerated
818,48.0,"Oranges, flesh only",48.3,"Asda, loose oranges",Pack size:,Purchase Date: 21.11.201 1,21.11.201 1,NaN,Best Before Date:,Batch Number:,Packaging: Plastic,Plastic,Ambient,Ambient
819,48.0,"Oranges, flesh only",48.4,"Asda, medium Navel oranges",NaN,6 fruits,21.11.201 1,"Asda, Norwich",Best Before Date:,Batch Number: DP318 PO148551 900202 105530121075,DP318 PO148551 900202 105530121075,Netting,Ambient,Ambient
820,48.0,"Oranges, flesh only",48.6,"Morrisons, jumbo oranges, extra juicy",Morrisons,Purchase Date: 21.11.201 1,21.11.201 1,"Morrisons, Norwich",24-Nov,27-Nov,26205B TH1 CIT2B 323 1107:48 IS Cal: 3,Netting,Ambient,Ambient
821,48.0,"Oranges, flesh only",NaN,"The Co -operative, oranges",The Co -operative,4 fruit,21.11.201 1,"The Co -operative, Wymondham",Best Before Date: 26 Nov,26-Nov,MO29 09: 56 1047 GIR11 KEPIII KEPII,Netting,Ambient,Ambient
822,48.0,"Oranges, flesh only",48.8,"Aldi, oranges",Aldi,6 fruit,21.11.201 1,"Aldi, Norwich",Best Before Date:,Batch Number: 4705 E58100 11:27 1049 L7G1 supp...,4705 E58100 11:27 1049 L7G1 supplier no: 255,Netting,NaN,Ambient
823,48.0,"Oranges, flesh only",48.9,"Sainsbury’s, basics, oranges",Sainsbury’s basics,5 fruit,21.11.201 1,NaN,29-Nov,29-Nov,CO163A,Netting,Ambient,Ambient


Match

In [122]:
# Define your fuzzy matching function
def fuzzy_matching_score(source, target):
    return fuzz.token_sort_ratio(source, target) 

# Create a function to update 'Pack size' based on the match
def update_pack_size(row):
    cartdf = row['Sub-sample Number']
    
    # Find the best match in 'a' dataframe
    matches = oranges[oranges['Sub-sample\rnumber'].apply(lambda x: fuzzy_matching_score(cartdf, x)) >= 80]
    
    if not matches.empty:
        # Get the matched weights and select the highest one
        matched_weights = matches['Average weight\rwhole fruit (g)']
        best_match_weight = matched_weights.max()
        return best_match_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'filtered_products'
rang['Pack size'] = rang.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\1928965564.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rang['Pack size'] = rang.apply(update_pack_size, axis=1)


In [123]:
rang

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
817,48.0,"Oranges, flesh only",48.1,"Tesco, large oranges",Tesco,271.3,21.11.201 1,"Tesco, Blue Boar Road, Norwich",25-Nov,27-Nov,WO:43258 MZ16 TC:33101889 16:52 XO2,Netting,Ambient,Refrigerated
818,48.0,"Oranges, flesh only",48.3,"Asda, loose oranges",Pack size:,224.9,21.11.201 1,NaN,Best Before Date:,Batch Number:,Packaging: Plastic,Plastic,Ambient,Ambient
819,48.0,"Oranges, flesh only",48.4,"Asda, medium Navel oranges",NaN,217.6,21.11.201 1,"Asda, Norwich",Best Before Date:,Batch Number: DP318 PO148551 900202 105530121075,DP318 PO148551 900202 105530121075,Netting,Ambient,Ambient
820,48.0,"Oranges, flesh only",48.6,"Morrisons, jumbo oranges, extra juicy",Morrisons,273.7,21.11.201 1,"Morrisons, Norwich",24-Nov,27-Nov,26205B TH1 CIT2B 323 1107:48 IS Cal: 3,Netting,Ambient,Ambient
821,48.0,"Oranges, flesh only",NaN,"The Co -operative, oranges",The Co -operative,4 fruit,21.11.201 1,"The Co -operative, Wymondham",Best Before Date: 26 Nov,26-Nov,MO29 09: 56 1047 GIR11 KEPIII KEPII,Netting,Ambient,Ambient
822,48.0,"Oranges, flesh only",48.8,"Aldi, oranges",Aldi,175.7,21.11.201 1,"Aldi, Norwich",Best Before Date:,Batch Number: 4705 E58100 11:27 1049 L7G1 supp...,4705 E58100 11:27 1049 L7G1 supplier no: 255,Netting,NaN,Ambient
823,48.0,"Oranges, flesh only",48.9,"Sainsbury’s, basics, oranges",Sainsbury’s basics,151.6,21.11.201 1,NaN,29-Nov,29-Nov,CO163A,Netting,Ambient,Ambient


In [124]:
rang.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
rang.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\Oranges.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\886200231.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rang.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


                                                        Melon

In [125]:
melon = saved_dataframes[73][:9]
melon

,Sub-sample\rnumber,Average weight\rwhole fruit (g),Average weight\rseeds (g),Average weight\rskin (g),Edible portion
0,49.1,1333.8,103.4,532.2,0.52
1,49.2,1289.3,121.1,528.2,0.50
2,49.3,1971.1,155.8,861.8,0.48
3,49.4,1340.9,136.6,470.9,0.55
4,49.5,1727.0,163.7,612.5,0.55
5,49.6,Purchased as ready to eat chunks,1.0,NaN,NaN
6,49.7,1515.2,146.6,565.1,0.53
7,49.8,1681.6,183.0,611.5,0.53
8,49.9,1218.8,168.6,388.9,0.54


In [126]:
mel = filtered_products[837:846]
mel

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
837,49.0,"Melon, yellow flesh, flesh only",49.1,"Tesco, Honeydew melon",Tesco,Purchase Date: 30.08.2011,30.08.2011,"Tesco, Harford Bridges, Norwich",Best Before Date:,Batch Number:,Packaging:,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
838,49.0,"Melon, yellow flesh, flesh only",49.2,"Tesco, Honeydew melon",Tesco,Purchase Date: 30.08.2011,30.08.2011,"Tesco, Harford Bridges, Norwich",Best Before Date:,Batch Number:,Packaging:,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
839,49.0,"Melon, yellow flesh, flesh only",49.3,"Asda, large Honeydew melon",Asda,NaN,30.08.2011,"Asda, Norwich",Best Before Date:,Batch Number: 905084,905084,None,Ambient,Refrigerated
840,49.0,"Melon, yellow flesh, flesh on ly",49.4,"Asda, Honeydew melon",Asda,Purchase Date: 30.08.2011,30.08.2011,"Asda, Norwich",Best Before Date:,Batch Number: 905084,905084,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
841,NaN,Sub-sample Number:,Sub-sample Name:,Brand:,Pack size:,Purchase Date:,Place of Purchase:,Display Until Date:,Best Before Date:,Batch Number:,Packaging:,Sample Condition As Purchased:,Post Purchase Storage:,49
842,49.0,"Melon, yellow flesh, flesh only",49.6,"Morrisons, Honeydew melon chunks",Morrisons,400g,30.08.2011,"Morrisons, Norwich",Use By Date: 01 Sep,NaN,00:44 1471 T32345/1110,Plastic,NaN,Refrigerated
843,49.0,"Melon, yellow flesh, flesh only",49.7,"Marks & Sp encer, perfectly ripe Honeydew melon",Marks & Spencer,Purchase Date: 30.08.2011,30.08.2011,"Marks & Spencer, Norwich",04/05 Sep,Batch Number: 1826,1826,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
844,49.0,"Melon, yellow flesh, flesh only",49.8,"Lidl, honeydew melon",Pack size:,Purchase Date: 30.08.2011,30.08.2011,"Lidl, Norwich",Best Before Date:,Batch Number:,Packaging:,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
845,49.0,"Melon, yellow flesh, flesh only",49.9,Market H oneydew melon,Pack size:,Purchase Date: 30.08.2011,30.08.2011,"Independent market stall, Norwich",Best Before Date:,Batch Number: C.I.F. B -54543269,C.I.F. B -54543269,Sample Condition As Purchased: Ambient,Ambient,Refrigerated


Match

In [127]:
# Define your fuzzy matching function
def fuzzy_matching_score(source, target):
    return fuzz.token_sort_ratio(source, target) 

# Create a function to update 'Pack size' based on the match
def update_pack_size(row):
    cartdf = row['Sub-sample Number']
    
    # Find the best match in 'a' dataframe
    matches = melon[melon['Sub-sample\rnumber'].apply(lambda x: fuzzy_matching_score(cartdf, x)) >= 80]
    
    if not matches.empty:
        # Get the matched weights and select the highest one
        matched_weights = matches['Average weight\rwhole fruit (g)']
        best_match_weight = matched_weights.max()
        return best_match_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'filtered_products'
mel['Pack size'] = mel.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\4134558841.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mel['Pack size'] = mel.apply(update_pack_size, axis=1)


In [128]:
mel

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
837,49.0,"Melon, yellow flesh, flesh only",49.1,"Tesco, Honeydew melon",Tesco,1333.8,30.08.2011,"Tesco, Harford Bridges, Norwich",Best Before Date:,Batch Number:,Packaging:,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
838,49.0,"Melon, yellow flesh, flesh only",49.2,"Tesco, Honeydew melon",Tesco,1289.3,30.08.2011,"Tesco, Harford Bridges, Norwich",Best Before Date:,Batch Number:,Packaging:,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
839,49.0,"Melon, yellow flesh, flesh only",49.3,"Asda, large Honeydew melon",Asda,1971.1,30.08.2011,"Asda, Norwich",Best Before Date:,Batch Number: 905084,905084,None,Ambient,Refrigerated
840,49.0,"Melon, yellow flesh, flesh on ly",49.4,"Asda, Honeydew melon",Asda,1340.9,30.08.2011,"Asda, Norwich",Best Before Date:,Batch Number: 905084,905084,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
841,NaN,Sub-sample Number:,Sub-sample Name:,Brand:,Pack size:,Purchase Date:,Place of Purchase:,Display Until Date:,Best Before Date:,Batch Number:,Packaging:,Sample Condition As Purchased:,Post Purchase Storage:,49
842,49.0,"Melon, yellow flesh, flesh only",49.6,"Morrisons, Honeydew melon chunks",Morrisons,Purchased as ready to eat chunks,30.08.2011,"Morrisons, Norwich",Use By Date: 01 Sep,NaN,00:44 1471 T32345/1110,Plastic,NaN,Refrigerated
843,49.0,"Melon, yellow flesh, flesh only",49.7,"Marks & Sp encer, perfectly ripe Honeydew melon",Marks & Spencer,1515.2,30.08.2011,"Marks & Spencer, Norwich",04/05 Sep,Batch Number: 1826,1826,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
844,49.0,"Melon, yellow flesh, flesh only",49.8,"Lidl, honeydew melon",Pack size:,1681.6,30.08.2011,"Lidl, Norwich",Best Before Date:,Batch Number:,Packaging:,Sample Condition As Purchased: Ambient,Ambient,Refrigerated
845,49.0,"Melon, yellow flesh, flesh only",49.9,Market H oneydew melon,Pack size:,1218.8,30.08.2011,"Independent market stall, Norwich",Best Before Date:,Batch Number: C.I.F. B -54543269,C.I.F. B -54543269,Sample Condition As Purchased: Ambient,Ambient,Refrigerated


In [129]:
mel.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
mel.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\Melon.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\3510771713.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mel.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


                                                    Peas Frozen

In [130]:
peasf = saved_dataframes[74][:9]
peasf

,Sub-sample\rnumber,Cooking instructions,Weight\rbefore\rcooking (g),Weight\rwater\radded (g),Cooked\rweight (g),% Weight\rchange
0,51.1,"Add 15 g water per 160g,\rcook 3 mins",168.5,16.1,165.1,-2.0
1,51.2,"Add 15 g water per 160g,\rcook 3 mins",161.2,16.1,151.2,-6.2
2,51.3,"Add 15 g water per 160g,\rcook 3 mins",162.0,16.6,156.0,-3.7
3,51.4,"Add 15 g water per 160g,\rcook 3 mins",168.6,16.5,162.6,-3.6
4,51.5,"Add 30g water per 225g,\rcook 4 mins, stir hal...",222.5,28.4,208.5,-6.3
5,51.6,"Add 30g water per 225g,\rcook 4 mins, stir hal...",224.3,31.7,207.8,-7.4
6,51.7,"Add 10 g water per 160g,\rcook 4 mins",159.2,11.2,130.2,-18.2
7,51.8,"Add 15g water per 100g,\rcook 2 mins, stir hal...",208.0,29.5,175.5,-15.6
8,51.9,"Add 30g water per 250g,\rcook 4.5 mins, stir\r...",253.0,30.7,247.2,-2.3


In [131]:
pea = filtered_products[864:873]
pea

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
864,NaN,"Peas, frozen, microwaved",51.1,"Birds Eye, field fresh garden peas",Birds Eye,480g,27.01.2012,"Sainsbury’s, Longwater , Norwich",Best Before Date: 05 2013,05 2013,L 1327 MJ 124 11:15 101498 87702161,Plastic,Frozen,Frozen
865,51.0,"Peas, frozen, microwaved",51.2,"Birds Eye, field fresh garden peas",Birds Eye,900g,27.01.2012,NaN,Best Before Date: 07 2013,07 2013,L 2013 GH 124 06:41 101504 87702162,Plastic,Frozen,Frozen
866,51.0,"Peas, frozen, microwaved",51.3,"Birds Eye, field fresh garden peas",Birds Eye,1.52kg,30.01.2012,3663 delivery,Best Before Date: 05 2013,05 2013,L1306 KY124 09:11 87702649,Plastic,Frozen,Frozen
867,51.0,"Peas, frozen, microwaved",51.4,"Birds Eye, field fresh petit pois",Birds Eye,700g,27.01.2012,"Sainsbury’s, Longwater, Norwich",Best Before Date: 06 2013,06 2013,L1347 NJ 124 12:44 105168 87703203,Foil,Frozen,NaN
868,51.0,"Peas, frozen, microwaved",51.5,"Tesco, garden peas",Tesco,1kg,30.01.2012,"Tesco, Blue Boar Road, Norwich",Oct-12,Jan-13,2024TMHA1 02:41,Plastic,Frozen,NaN
869,51.0,"Peas, fr ozen, microwaved",51.6,"Tesco, Market Value, garden peas",Tesco Market Value,900g,30.01.2012,"Tesco, Blue Boar Road, Norwich",Oct-12,Jan-13,2019TFK 4 10:55,Plastic,Frozen,Frozen
870,51.0,"Peas, frozen, microwaved",51.7,"Sainsbury’s SO Organic, garden peas",Sainsbury’s SO Organic,750g,27.01.2012,"Sainsbury’s, Lon gwater, Norwich",Best Before Date: Jun 2013,Jun-13,DF1347 07:16 1 6333630 10.10 Pl405,Plastic,Frozen,Frozen
871,51.0,"Peas, frozen, microwaved",51.8,"Asda, Chosen by you, garden peas",Asda Chosen by you,1kg,27.01.2012,"Asda, Norwich",Best Before Date: 26 Feb 13,26-Feb-13,1 241 12:02,Plastic,Frozen,Frozen
872,51.0,"Peas, frozen, microwaved",51.9,"Morrisons, petit pois",Morrisons,454g,30.01.2012,"Morrisons, Norwich",Best Before Date: Jun 2013,Jun-13,L1336B1 BE22 11:50 2332 T31686/0810,Plastic,Frozen,Frozen


Match

In [132]:
# Define your fuzzy matching function
def fuzzy_matching_score(source, target):
    return fuzz.token_sort_ratio(source, target) 

# Create a function to update 'Pack size' based on the match
def update_pack_size(row):
    cartdf = row['Sub-sample Number']
    
    # Find the best match in 'a' dataframe
    matches = peasf[peasf['Sub-sample\rnumber'].apply(lambda x: fuzzy_matching_score(cartdf, x)) >= 80]
    
    if not matches.empty:
        # Get the matched weights and select the highest one
        matched_weights = matches['Weight\rbefore\rcooking (g)']
        best_match_weight = matched_weights.max()
        return best_match_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'filtered_products'
pea['Pack size'] = pea.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\2713959851.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pea['Pack size'] = pea.apply(update_pack_size, axis=1)


In [133]:
pea

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
864,NaN,"Peas, frozen, microwaved",51.1,"Birds Eye, field fresh garden peas",Birds Eye,168.5,27.01.2012,"Sainsbury’s, Longwater , Norwich",Best Before Date: 05 2013,05 2013,L 1327 MJ 124 11:15 101498 87702161,Plastic,Frozen,Frozen
865,51.0,"Peas, frozen, microwaved",51.2,"Birds Eye, field fresh garden peas",Birds Eye,161.2,27.01.2012,NaN,Best Before Date: 07 2013,07 2013,L 2013 GH 124 06:41 101504 87702162,Plastic,Frozen,Frozen
866,51.0,"Peas, frozen, microwaved",51.3,"Birds Eye, field fresh garden peas",Birds Eye,162.0,30.01.2012,3663 delivery,Best Before Date: 05 2013,05 2013,L1306 KY124 09:11 87702649,Plastic,Frozen,Frozen
867,51.0,"Peas, frozen, microwaved",51.4,"Birds Eye, field fresh petit pois",Birds Eye,168.6,27.01.2012,"Sainsbury’s, Longwater, Norwich",Best Before Date: 06 2013,06 2013,L1347 NJ 124 12:44 105168 87703203,Foil,Frozen,NaN
868,51.0,"Peas, frozen, microwaved",51.5,"Tesco, garden peas",Tesco,222.5,30.01.2012,"Tesco, Blue Boar Road, Norwich",Oct-12,Jan-13,2024TMHA1 02:41,Plastic,Frozen,NaN
869,51.0,"Peas, fr ozen, microwaved",51.6,"Tesco, Market Value, garden peas",Tesco Market Value,224.3,30.01.2012,"Tesco, Blue Boar Road, Norwich",Oct-12,Jan-13,2019TFK 4 10:55,Plastic,Frozen,Frozen
870,51.0,"Peas, frozen, microwaved",51.7,"Sainsbury’s SO Organic, garden peas",Sainsbury’s SO Organic,159.2,27.01.2012,"Sainsbury’s, Lon gwater, Norwich",Best Before Date: Jun 2013,Jun-13,DF1347 07:16 1 6333630 10.10 Pl405,Plastic,Frozen,Frozen
871,51.0,"Peas, frozen, microwaved",51.8,"Asda, Chosen by you, garden peas",Asda Chosen by you,208.0,27.01.2012,"Asda, Norwich",Best Before Date: 26 Feb 13,26-Feb-13,1 241 12:02,Plastic,Frozen,Frozen
872,51.0,"Peas, frozen, microwaved",51.9,"Morrisons, petit pois",Morrisons,253.0,30.01.2012,"Morrisons, Norwich",Best Before Date: Jun 2013,Jun-13,L1336B1 BE22 11:50 2332 T31686/0810,Plastic,Frozen,Frozen


In [134]:
pea.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
pea.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\peasf.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\3606900008.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pea.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


                                                   Tomato, canned

In [135]:
cantomato = saved_dataframes[75][:9]
cantomato

,Sub-sample\rnumber,Heating time\r(min),Weight before\rheating (g),Weight after\rheating (g),% Weight loss
0,53.1,3.0,428.9,391.4,8.7
1,53.2,2.0,239.0,216.9,9.2
2,53.3,3.0,403.4,393.6,2.4
3,53.4,3.0,408.6,380.9,6.8
4,53.5,4.0,269.4,252.4,6.3
5,53.6,4.0,399.7,377.2,5.6
6,53.7,3.5,379.0,360.2,5.0
7,53.8,3.0,395.2,378.0,4.4
8,53.9,3.0,399.7,382.8,4.2


In [136]:
ctoma = filtered_products[886:894]
ctoma

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
886,53.0,"Tomatoes, canned, whole contents",53.1,"Napolina, chopped tomatoes in a rich tomato ju...",Napolina,400g (net weight),27.01.2012,"Asda, Norwich",NaN,Dec-13,CO5 LE216 PC,Metal,Ambient,Ambient
887,53.0,"Tomatoes, canned, whole contents",53.2,"Napo lina, chopped tomatoes in a rich tomato j...",Napolina,400g (net),30.01.2012,"Tesco, Blue Boar Road, Norwich",Best Before Date: Dec 2013,Dec-13,CO5 LE213 PC,Metal,Ambient,Ambient
888,53.0,"Tomatoes, canned, whole contents",53.3,"Napolina, peeled plum tomatoes in a rich tomat...",Napolina,400g,27.01.2012,"Booker, Norwich",Best Before Date: Dec 2011,Dec-11,CO5 7 PP L R239 12:18,Metal,Ambient,Ambient
889,53.0,"Tomatoes, canned, whole contents",53.4,"Cirio, peeled plum tomatoes",Cirio,"400g (net weight), 250g (drained weight)",27.01.2012,"Sainsbury’ s, Longwater, Norwich",Best Before Date: 31/12/2014,31/12/2014,L E241 15:46 J10 PPA,Metal,Ambient,Ambient
890,53.0,"Tomatoes, ca nned, whole contents",53.5,"Tesco, Italian chopped tomatoes in rich tomato...",Tesco,400g,30.01.2012,"Tesco, Blue Boar Road, Norwich",Best Before Date: 07/2013,Jul-13,FD1 C55 LE208,Metal,Ambient,Ambient
891,53.0,"Tomatoes, canned, whole contents",53.6,"Tesco Value, peeled plum tomatoes in tomato ju...",Tesco Value,400g,30.01.2012,"Tesco, Blue Boar Road, Norwich",Best Before Date: Dec 2012,Dec-12,EUM1 PO16 LN263,Metal,Ambient,Ambient
892,NaN,"Tomatoes, canned, whole contents",53.7,"Sainsbury’s, chopped tomatoes in tomato juice",Sainsbury’s,390g,27.01.2012,"Sainsbury’s, Longwater, Norwich",Best Before Date: Sep 2013,Sep-13,FF1 LE 267 15:54 06365335 06.10 D0104,Tetra pak,Ambient,Ambient
893,53.0,"Tomatoes, canned, whole contents",53.9,"Morrisons, peeled Italian plum tomatoes in tom...",Morrisons,"400g, 240g (drained)",30.01.2012,"Morrisons, Norwich",Best Before Date: Sep 2012,Sep-12,CO5 LV245 PP P01/11 80854 2605,Metal,Ambient,NaN


Match

In [137]:
# Define your fuzzy matching function
def fuzzy_matching_score(source, target):
    return fuzz.token_sort_ratio(source, target) 

# Create a function to update 'Pack size' based on the match
def update_pack_size(row):
    cartdf = row['Sub-sample Number']
    
    # Find the best match in 'a' dataframe
    matches = cantomato[cantomato['Sub-sample\rnumber'].apply(lambda x: fuzzy_matching_score(cartdf, x)) >= 80]
    
    if not matches.empty:
        # Get the matched weights and select the highest one
        matched_weights = matches['Weight before\rheating (g)']
        best_match_weight = matched_weights.max()
        return best_match_weight
    
    return row['Pack size']  # If no match, return the original 'Pack size'

# Apply the update_pack_size function to 'filtered_products'
ctoma['Pack size'] = ctoma.apply(update_pack_size, axis=1)

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\3819065961.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctoma['Pack size'] = ctoma.apply(update_pack_size, axis=1)


In [138]:
ctoma

,Composite Sample Number,Composite Sample Name,Sub-sample Number,Sub-sample Name,Brand,Pack size,Purchase Date,Place of Purchase,Display Until Date,Best Before Date,Batch Number,Packaging,Sample Condition As Purchased,Post Purchase Storage
886,53.0,"Tomatoes, canned, whole contents",53.1,"Napolina, chopped tomatoes in a rich tomato ju...",Napolina,428.9,27.01.2012,"Asda, Norwich",NaN,Dec-13,CO5 LE216 PC,Metal,Ambient,Ambient
887,53.0,"Tomatoes, canned, whole contents",53.2,"Napo lina, chopped tomatoes in a rich tomato j...",Napolina,239.0,30.01.2012,"Tesco, Blue Boar Road, Norwich",Best Before Date: Dec 2013,Dec-13,CO5 LE213 PC,Metal,Ambient,Ambient
888,53.0,"Tomatoes, canned, whole contents",53.3,"Napolina, peeled plum tomatoes in a rich tomat...",Napolina,403.4,27.01.2012,"Booker, Norwich",Best Before Date: Dec 2011,Dec-11,CO5 7 PP L R239 12:18,Metal,Ambient,Ambient
889,53.0,"Tomatoes, canned, whole contents",53.4,"Cirio, peeled plum tomatoes",Cirio,408.6,27.01.2012,"Sainsbury’ s, Longwater, Norwich",Best Before Date: 31/12/2014,31/12/2014,L E241 15:46 J10 PPA,Metal,Ambient,Ambient
890,53.0,"Tomatoes, ca nned, whole contents",53.5,"Tesco, Italian chopped tomatoes in rich tomato...",Tesco,269.4,30.01.2012,"Tesco, Blue Boar Road, Norwich",Best Before Date: 07/2013,Jul-13,FD1 C55 LE208,Metal,Ambient,Ambient
891,53.0,"Tomatoes, canned, whole contents",53.6,"Tesco Value, peeled plum tomatoes in tomato ju...",Tesco Value,399.7,30.01.2012,"Tesco, Blue Boar Road, Norwich",Best Before Date: Dec 2012,Dec-12,EUM1 PO16 LN263,Metal,Ambient,Ambient
892,NaN,"Tomatoes, canned, whole contents",53.7,"Sainsbury’s, chopped tomatoes in tomato juice",Sainsbury’s,379.0,27.01.2012,"Sainsbury’s, Longwater, Norwich",Best Before Date: Sep 2013,Sep-13,FF1 LE 267 15:54 06365335 06.10 D0104,Tetra pak,Ambient,Ambient
893,53.0,"Tomatoes, canned, whole contents",53.9,"Morrisons, peeled Italian plum tomatoes in tom...",Morrisons,399.7,30.01.2012,"Morrisons, Norwich",Best Before Date: Sep 2012,Sep-12,CO5 LV245 PP P01/11 80854 2605,Metal,Ambient,NaN


In [139]:
ctoma.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)
ctoma.to_csv(r"C:\Users\medekar\Desktop\Product_Weight_Project\Code\Jupter_notebooks\Product Weights\Cannedtomatoes.csv")

C:\Users\medekar\AppData\Local\Temp\ipykernel_19056\2225110645.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ctoma.rename(columns={'Pack size': 'Sample Weight'}, inplace = True)


                                                        Next Notebook